### 获取所需pull_request数据

In [1]:
from odps import ODPS
from odps import options
from odps.df import DataFrame
import pandas as pd
import numpy as np

# ACCESS_ID SECRET_ACCESS_KEYvODPS_PROJECT ODPS_ENDPOINT github提示有敏感信息无法上传，数据集来源为阿里云
# 创建odps实例
o = ODPS(ACCESS_ID, SECRET_ACCESS_KEY,
         project=ODPS_PROJECT, endpoint=ODPS_ENDPOINT)
options.tunnel.limit_instance_tunnel = False

In [2]:
TEMP_TABLE_NAME1 = 'hx_repo_actor_pull_request'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        repo_id     BIGINT,
        actor_id    BIGINT,
        new_created_at    DATETIME,
        count    BIGINT
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME1
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520014839669g8jbakm7>

In [3]:
# 把时间改成16年到20年，把带bot的actor去掉
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME}
        SELECT
            repo_id,
            actor_id,
            TO_DATE(SUBSTRING_INDEX(created_at,' ',1), 'yyyy-mm-dd') AS new_created_at,
            COUNT(*) AS count
        FROM ods_github_log
        WHERE (type = 'PullRequestEvent') AND (action = 'opened')
        AND pt>='20160101'
        AND pt<'20220101'
        AND actor_id IN (
            SELECT actor_id
            FROM ods_github_log
            WHERE (type = 'PullRequestEvent') AND (action = 'opened')
            AND pt>='20160101'
            AND pt<'20220101'
            GROUP BY actor_id
            HAVING COUNT(*) > 10
        )
        AND actor_login NOT LIKE '%bot%'
        GROUP BY repo_id, actor_id, new_created_at
        ORDER BY repo_id, actor_id, new_created_at'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME1)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [4]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME1))
data.head(10)

,repo_id,actor_id,new_created_at,count
0,1,24193253,2020-10-01,1
1,27,85,2016-02-05,1
2,27,85,2016-02-06,1
3,27,85,2016-04-22,1
4,27,85,2016-04-25,1
5,27,85,2016-05-01,2
6,27,85,2016-05-06,1
7,27,85,2016-05-21,1
8,27,85,2016-06-05,1
9,27,85,2016-06-19,1


#### 查看数据量

In [ ]:
# select_distinct_repo_id_sql = '''
#     SELECT DISTINCT repo_id
#     FROM {TEMP_TABLE_NAME}
# '''.format(TEMP_TABLE_NAME = TEMP_TABLE_NAME1)
# table_distinct_repo_id = o.execute_sql(select_distinct_repo_id_sql, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})

In [ ]:
# cnt = 0
# with table_distinct_repo_id.open_reader(tunnel=True) as reader:
#     for record in reader:
#         cnt = cnt + 1
# print(cnt)

In [ ]:
# select_distinct_actor_id_sql = '''
#     SELECT DISTINCT actor_id
#     FROM {TEMP_TABLE_NAME}
# '''.format(TEMP_TABLE_NAME = TEMP_TABLE_NAME1)
# table_distinct_actor_id = o.execute_sql(select_distinct_actor_id_sql, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})

In [ ]:
# cnt = 0
# with table_distinct_actor_id.open_reader(tunnel=True) as reader:
#     for record in reader:
#         cnt = cnt + 1
# print(cnt)

同一repo同一actor只pull过2次的

In [ ]:
# TEMP_TABLE_NAME111 = 'hx_repo_actor_pull_request_record_num_is_2'
# drop_sql = '''
#     DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
#     CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
#     (
#         repo_id     BIGINT,
#         actor_id    BIGINT
#     );
# '''.format(
#     TEMP_TABLE_NAME=TEMP_TABLE_NAME111
# )
# o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

In [ ]:
# record_num_is_2_sql = '''set odps.sql.validate.orderby.limit=false;
#     INSERT INTO {TEMP_TABLE_NAME2}
#     SELECT repo_id, actor_id
#     FROM {TEMP_TABLE_NAME1}
#     GROUP BY repo_id, actor_id
#     HAVING count(*) = 2
# '''.format(
#     TEMP_TABLE_NAME1 = TEMP_TABLE_NAME1,
#     TEMP_TABLE_NAME2 = TEMP_TABLE_NAME111)
# table = o.execute_sql(record_num_is_2_sql, hints={"odps.sql.submit.mode" : "script"})

In [ ]:
# data = DataFrame(o.get_table(TEMP_TABLE_NAME111))
# data.head(10)

### weight值的计算

In [5]:
TEMP_TABLE_NAME2 = 'hx_repo_actor_pull_request_next'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        repo_id     BIGINT,
        actor_id    BIGINT,
        new_created_at    DATETIME,
        next_created_at    DATETIME
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME2
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520015705549gp2r7om7>

In [6]:
insert_next_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME2}
        SELECT
            repo_id,
            actor_id,
            new_created_at,
            lead(new_created_at,1) over(PARTITION BY repo_id, actor_id ORDER BY new_created_at) AS next_created_at
        FROM {TEMP_TABLE_NAME1}
'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME1,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME2
)
table = o.execute_sql(insert_next_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [7]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME2))
data.head(15)

,repo_id,actor_id,new_created_at,next_created_at
0,1,24193253,2020-10-01,NaT
1,27,85,2016-02-05,2016-02-06
2,27,85,2016-02-06,2016-04-22
3,27,85,2016-04-22,2016-04-25
4,27,85,2016-04-25,2016-05-01
5,27,85,2016-05-01,2016-05-06
6,27,85,2016-05-06,2016-05-21
7,27,85,2016-05-21,2016-06-05
8,27,85,2016-06-05,2016-06-19
9,27,85,2016-06-19,2016-07-02


In [8]:
TEMP_TABLE_NAME3 = 'hx_repo_actor_pull_request_diff'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        repo_id     BIGINT,
        actor_id    BIGINT,
        datediff    BIGINT
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME3
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 2022052002013772g3ts46pr2>

In [9]:
insert_diff_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME3}
        SELECT
            repo_id,
            actor_id,
            datediff(next_created_at, new_created_at, 'dd')
        FROM {TEMP_TABLE_NAME2}
'''.format(
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME2,
    TEMP_TABLE_NAME3=TEMP_TABLE_NAME3
)
table = o.execute_sql(insert_diff_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [10]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME3))
data.head(15)

,repo_id,actor_id,datediff
0,1,24193253,NaN
1,27,85,1.0
2,27,85,76.0
3,27,85,3.0
4,27,85,6.0
5,27,85,5.0
6,27,85,15.0
7,27,85,15.0
8,27,85,14.0
9,27,85,13.0


In [11]:
TEMP_TABLE_NAME4 = 'hx_repo_actor_pull_request_mean_and_var'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        repo_id     BIGINT,
        actor_id    BIGINT,
        mean    DOUBLE,
        var    DOUBLE
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME4
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520020239184gysv451>

In [12]:
insert_mean_and_var_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME4}
        SELECT
            repo_id,
            actor_id,
            AVG(datediff) as mean,
            STDDEV(datediff) as var
        FROM {TEMP_TABLE_NAME3}
        GROUP BY repo_id, actor_id
'''.format(
    TEMP_TABLE_NAME3=TEMP_TABLE_NAME3,
    TEMP_TABLE_NAME4=TEMP_TABLE_NAME4
)
table = o.execute_sql(insert_mean_and_var_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [13]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME4))
data.head(15)

,repo_id,actor_id,mean,var
0,1,24193253,NaN,NaN
1,27,85,16.444444,21.674927
2,27,10416,NaN,NaN
3,27,16881,NaN,NaN
4,27,18191,107.000000,106.000000
5,27,25555,NaN,NaN
6,27,39406,NaN,NaN
7,27,43569,NaN,NaN
8,27,48936,NaN,NaN
9,27,79817,4.000000,0.000000


In [14]:
TEMP_TABLE_NAME5 = 'hx_repo_actor_pull_request_burst'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        repo_id     BIGINT,
        actor_id    BIGINT,
        burst    DOUBLE
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME5
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520020356420gulwdgr>

In [15]:
insert_burst_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME5}
        SELECT
            repo_id,
            actor_id,
            (var - mean) / (var + mean) AS burst
        FROM {TEMP_TABLE_NAME4}
'''.format(
    TEMP_TABLE_NAME4=TEMP_TABLE_NAME4,
    TEMP_TABLE_NAME5=TEMP_TABLE_NAME5
)
table = o.execute_sql(insert_burst_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [16]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME5))
data.head(15)

,repo_id,actor_id,burst
0,1,24193253,NaN
1,27,85,0.137213
2,27,10416,NaN
3,27,16881,NaN
4,27,18191,-0.004695
5,27,25555,NaN
6,27,39406,NaN
7,27,43569,NaN
8,27,48936,NaN
9,27,79817,-1.000000


查看actor只给repo pull过一次的

In [ ]:
# select_num_2_sql = '''
#     SELECT * FROM {TEMP_TABLE_NAME} WHERE repo_id = 3314 AND actor_id = 27856297
# '''.format(TEMP_TABLE_NAME=TEMP_TABLE_NAME5)
# table_num_2 = o.execute_sql(select_num_2_sql, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})

In [ ]:
# with table_num_2.open_reader(tunnel=True) as reader:
#     for record in reader:
#         print(record)

计算weight

In [17]:
TEMP_TABLE_NAME6 = 'hx_repo_actor_pull_request_weight'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        repo_id     BIGINT,
        actor_id    BIGINT,
        weight    DOUBLE
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME6
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520020405169gpryk4pr2>

In [18]:
insert_weight_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME6}
        SELECT
            repo_id,
            actor_id,
            - burst AS weight
        FROM {TEMP_TABLE_NAME5}
'''.format(
    TEMP_TABLE_NAME5=TEMP_TABLE_NAME5,
    TEMP_TABLE_NAME6=TEMP_TABLE_NAME6
)
table = o.execute_sql(insert_weight_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [19]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME6)).fillna(0)
data.head(15)

,repo_id,actor_id,weight
0,1,24193253,0.000000
1,27,85,-0.137213
2,27,10416,0.000000
3,27,16881,0.000000
4,27,18191,0.004695
5,27,25555,0.000000
6,27,39406,0.000000
7,27,43569,0.000000
8,27,48936,0.000000
9,27,79817,1.000000


以上，得到的odps.DataFrame的fillna的结果

以下，对于直接把odps.DataFrame转换为pd.df的测试，很快

In [20]:
pd_data = data.to_pandas()
pd_data.head()

,repo_id,actor_id,weight
0,1,24193253,0.000000
1,27,85,-0.137213
2,27,10416,0.000000
3,27,16881,0.000000
4,27,18191,0.004695


看看一共有多少数据

In [ ]:
# select_sql = '''
#     SELECT * FROM {TEMP_TABLE_NAME}
# '''.format(TEMP_TABLE_NAME=TEMP_TABLE_NAME5)
# table_weight = o.execute_sql(select_sql, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})

In [ ]:
# cnt = 0
# with table_weight.open_reader(tunnel=True) as reader:
#     for record in reader:
#         cnt = cnt + 1
# print(cnt)

### repo_id，actor_id重新编号

#### 将repo_id和actor_id映射到较小的范围里

In [21]:
TEMP_TABLE_NAME7 = 'hx_repo_actor_pull_request_new_repoid_pre'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        repo_id     BIGINT,
        all_zero    BIGINT DEFAULT 0
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME7
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520021821501gacd4ssa>

In [22]:
insert_repoid_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME2}(repo_id)
        SELECT DISTINCT repo_id
        FROM {TEMP_TABLE_NAME1}
'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME6,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME7
)
table = o.execute_sql(insert_repoid_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [23]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME7))
data.head(15)

,repo_id,all_zero
0,1,0
1,27,0
2,28,0
3,35,0
4,43,0
5,68,0
6,130,0
7,131,0
8,142,0
9,144,0


In [24]:
TEMP_TABLE_NAME8 = 'hx_repo_actor_pull_request_new_repoid'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        repo_id     BIGINT,
        new_repo_id    BIGINT
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME8
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520021830558gwm1ggr>

In [25]:
insert_new_repoid_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME2}
        SELECT repo_id, row_number() over(partition by all_zero order by repo_id) - 1 as new_repo_id
        FROM {TEMP_TABLE_NAME1}
'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME7,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME8
)
table = o.execute_sql(insert_new_repoid_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [26]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME8))
data.head(15)

,repo_id,new_repo_id
0,1,0
1,27,1
2,28,2
3,35,3
4,43,4
5,68,5
6,130,6
7,131,7
8,142,8
9,144,9


In [27]:
# 查看最大new_repo_id(用于保证与watch中的new_repo_id一致)
select_max_new_repo_id = '''SELECT MAX(new_repo_id) FROM {TEMP_WATCH_TABLE}'''.format(TEMP_WATCH_TABLE=TEMP_TABLE_NAME8)
table_max_new_repo_id = o.execute_sql(select_max_new_repo_id, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})
with table_max_new_repo_id.open_reader(tunnel=True) as reader:
     for record in reader:
            print(record)

odps.Record {
  _c0   9021971
}


new_repoid完毕

以下，new_actorid

In [28]:
TEMP_TABLE_NAME9 = 'hx_repo_actor_pull_request_new_actorid_pre'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        actor_id     BIGINT,
        all_zero    BIGINT DEFAULT 0
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME9
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520021839764gafwuwy9>

In [29]:
insert_actorid_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME2}(actor_id)
        SELECT DISTINCT actor_id
        FROM {TEMP_TABLE_NAME1}
'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME6,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME9
)
table = o.execute_sql(insert_actorid_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [30]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME9))
data.head(15)

,actor_id,all_zero
0,1,0
1,4,0
2,7,0
3,17,0
4,18,0
5,19,0
6,20,0
7,21,0
8,22,0
9,25,0


In [31]:
TEMP_TABLE_NAME10 = 'hx_repo_actor_pull_request_new_actorid'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        actor_id     BIGINT,
        new_actor_id    BIGINT
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME10
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520021849360g4y0tj0a>

In [32]:
insert_new_actorid_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME2}
        SELECT actor_id, row_number() over(partition by all_zero order by actor_id) - 1 as new_actor_id
        FROM {TEMP_TABLE_NAME1}
'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME9,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME10
)
table = o.execute_sql(insert_new_actorid_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [33]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME10))
data.head(15)

,actor_id,new_actor_id
0,1,0
1,4,1
2,7,2
3,17,3
4,18,4
5,19,5
6,20,6
7,21,7
8,22,8
9,25,9


就此，有了两张中间表：

TEMP_TABLE_NAME8  (new_repoid)

TEMP_TABLE_NAME10  (new_actorid)

外加weight表TEMP_TABLE_NAME6

#### 将new_repo_id表、new_actor_id表与原表连接

##### 带有原本actor_id和repo_id的

In [ ]:
# TEMP_TABLE_NAME11 = 'hx_repo_actor_pull_request_new_repo_actor_id'
# drop_sql = '''
#     DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
#     CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
#     (
#         repo_id    BIGINT,
#         actor_id     BIGINT,
#         weight    DOUBLE,
#         new_repo_id    BIGINT,
#         new_actor_id     BIGINT
#     );
# '''.format(
#     TEMP_TABLE_NAME=TEMP_TABLE_NAME11
# )
# o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

In [ ]:
# insert_new_repo_actor_id_sql = '''set odps.sql.validate.orderby.limit=false;
#         INSERT INTO {TEMP_TABLE_NAME4}
#         SELECT {TEMP_TABLE_NAME1}.repo_id, {TEMP_TABLE_NAME1}.actor_id, {TEMP_TABLE_NAME1}.weight, {TEMP_TABLE_NAME2}.new_repo_id, {TEMP_TABLE_NAME3}.new_actor_id
#         FROM {TEMP_TABLE_NAME1}, {TEMP_TABLE_NAME2}, {TEMP_TABLE_NAME3}
#         WHERE {TEMP_TABLE_NAME1}.repo_id = {TEMP_TABLE_NAME2}.repo_id AND {TEMP_TABLE_NAME1}.actor_id = {TEMP_TABLE_NAME3}.actor_id
#         ORDER BY {TEMP_TABLE_NAME1}.repo_id, {TEMP_TABLE_NAME1}.actor_id
# '''.format(
#     TEMP_TABLE_NAME1=TEMP_TABLE_NAME6,
#     TEMP_TABLE_NAME2=TEMP_TABLE_NAME8,
#     TEMP_TABLE_NAME3=TEMP_TABLE_NAME10,
#     TEMP_TABLE_NAME4=TEMP_TABLE_NAME11
# )
# table = o.execute_sql(insert_new_repo_actor_id_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [ ]:
# data = DataFrame(o.get_table(TEMP_TABLE_NAME11))
# data.head(15)

##### 不带原本actor_id和repo_id的

In [34]:
TEMP_TABLE_NAME12 = 'hx_repo_actor_pull_request_new_repo_actor_id_clear'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        new_repo_id    BIGINT,
        new_actor_id     BIGINT,
        weight    DOUBLE       
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME12
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520021853462gimy54pr2>

In [35]:
insert_new_repo_actor_id_clear_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME4}
        SELECT {TEMP_TABLE_NAME2}.new_repo_id, {TEMP_TABLE_NAME3}.new_actor_id, {TEMP_TABLE_NAME1}.weight
        FROM {TEMP_TABLE_NAME1}, {TEMP_TABLE_NAME2}, {TEMP_TABLE_NAME3}
        WHERE {TEMP_TABLE_NAME1}.repo_id = {TEMP_TABLE_NAME2}.repo_id AND {TEMP_TABLE_NAME1}.actor_id = {TEMP_TABLE_NAME3}.actor_id
        ORDER BY {TEMP_TABLE_NAME2}.new_repo_id, {TEMP_TABLE_NAME3}.new_actor_id
'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME6,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME8,
    TEMP_TABLE_NAME3=TEMP_TABLE_NAME10,
    TEMP_TABLE_NAME4=TEMP_TABLE_NAME12
)
table = o.execute_sql(insert_new_repo_actor_id_clear_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [36]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME12)).fillna(1)
data.head(15)

,new_repo_id,new_actor_id,weight
0,0,674742,1.000000
1,1,28,-0.137213
2,1,3048,1.000000
3,1,4451,1.000000
4,1,4737,0.004695
5,1,6223,1.000000
6,1,8601,1.000000
7,1,9375,1.000000
8,1,10374,1.000000
9,1,15395,1.000000


In [37]:
new_pd_data = data.to_pandas()
new_pd_data.head()

,new_repo_id,new_actor_id,weight
0,0,674742,1.000000
1,1,28,-0.137213
2,1,3048,1.000000
3,1,4451,1.000000
4,1,4737,0.004695


##### 查看数据量

In [ ]:
# select_sql = '''
#     SELECT * FROM {TEMP_TABLE_NAME}
# '''.format(TEMP_TABLE_NAME=TEMP_TABLE_NAME12)
# table_new_weight = o.execute_sql(select_sql, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})

In [ ]:
# cnt = 0
# with table_new_weight.open_reader(tunnel=True) as reader:
#     for record in reader:
#         cnt = cnt + 1
# print(cnt)

### repo与actor的度的计算

从TEMP_TABLE_NAME12表中，抽取取值不同的repo_id和其count，actor同

#### repo的度

In [38]:
TEMP_TABLE_NAME13 = 'hx_repo_actor_pull_request_new_repo_degree'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        new_repo_id    BIGINT,
        repo_degree    BIGINT       
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME13
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520022447801gu98n4pr2>

In [39]:
insert_repo_degree_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME2}
        SELECT new_repo_id, count(*) AS repo_degree
        FROM {TEMP_TABLE_NAME1} 
        GROUP BY new_repo_id
        ORDER BY new_repo_id
'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME12,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME13
)
table = o.execute_sql(insert_repo_degree_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [40]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME13))
data.head(15)

,new_repo_id,repo_degree
0,0,1
1,1,33
2,2,6
3,3,1
4,4,1
5,5,29
6,6,80
7,7,1
8,8,1
9,9,15


In [41]:
repo_degree_pd_data = data.to_pandas()
repo_degree_pd_data.head()

,new_repo_id,repo_degree
0,0,1
1,1,33
2,2,6
3,3,1
4,4,1


##### repo个数

In [42]:
select_repo_degree_sql = '''
    SELECT * FROM {TEMP_TABLE_NAME}
'''.format(TEMP_TABLE_NAME=TEMP_TABLE_NAME13)
table_repo_degree = o.execute_sql(select_repo_degree_sql, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})

#### actor的度

In [43]:
TEMP_TABLE_NAME14 = 'hx_repo_actor_pull_request_new_actor_degree'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        new_actor_id    BIGINT,
        actor_degree    BIGINT       
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME14
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520022723836gmgmzwy9>

In [44]:
insert_actor_degree_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME2}
        SELECT new_actor_id, count(*) AS actor_degree
        FROM {TEMP_TABLE_NAME1} 
        GROUP BY new_actor_id
        ORDER BY new_actor_id
'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME12,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME14
)
table = o.execute_sql(insert_actor_degree_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [45]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME14))
data.head(15)

,new_actor_id,actor_degree
0,0,11
1,1,29
2,2,42
3,3,20
4,4,8
5,5,8
6,6,8
7,7,30
8,8,16
9,9,14


In [46]:
actor_degree_pd_data = data.to_pandas()
actor_degree_pd_data.head()

,new_actor_id,actor_degree
0,0,11
1,1,29
2,2,42
3,3,20
4,4,8


repo_degree_pd_data为pd.DataFrame类型，存储了仓库的度

actor_degree_pd_data为pd.DataFrame类型，存储了开发者的度

### name---id对应表

#### new_repo_id---repo_id---repo_name对应表

In [47]:
TEMP_TABLE_NAME15_1 = 'hx_repo_actor_pull_request_repo_withdate'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        repo_id     BIGINT,
        repo_name    STRING,
        created_at    DATETIME
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME15_1
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520022748892g7c005pr2>

In [48]:
repo_name_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME}
        SELECT
            repo_id,
            repo_name,
            created_at
        FROM ods_github_log
        WHERE (type = 'PullRequestEvent') AND (action = 'opened')
        AND pt>='20160101'
        AND pt<'20220101'
        AND actor_id IN (
            SELECT actor_id
            FROM ods_github_log
            WHERE (type = 'PullRequestEvent') AND (action = 'opened')
            AND pt>='20160101'
            AND pt<'20220101'
            GROUP BY actor_id
            HAVING COUNT(*) > 10
        )
        ORDER BY repo_id
'''.format(TEMP_TABLE_NAME=TEMP_TABLE_NAME15_1)
table = o.execute_sql(repo_name_sql, hints={"odps.sql.submit.mode" : "script"})

In [49]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME15_1))
data.head(10)

,repo_id,repo_name,created_at
0,1,mojombo/grit,2020-10-01 11:55:32
1,27,rubinius/rubinius,2016-01-21 03:07:33
2,27,rubinius/rubinius,2016-01-21 08:39:55
3,27,rubinius/rubinius,2016-01-22 09:53:16
4,27,rubinius/rubinius,2016-01-28 18:04:42
5,27,rubinius/rubinius,2016-10-09 03:06:09
6,27,rubinius/rubinius,2016-04-06 12:22:55
7,27,rubinius/rubinius,2016-07-23 03:55:02
8,27,rubinius/rubinius,2016-12-29 04:40:36
9,27,rubinius/rubinius,2016-07-23 03:02:06


In [50]:
TEMP_TABLE_NAME15_2 = 'hx_repo_actor_pull_request_repo_maxdate'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        repo_id     BIGINT,
        created_at    DATETIME
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME15_2
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520023537816gt2q7om7>

In [51]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME2}
        SELECT
            repo_id,
            MAX(created_at)
        FROM {TEMP_TABLE_NAME1}
        GROUP BY repo_id
        ORDER BY repo_id'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME15_1,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME15_2
)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [52]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME15_2))
data.head(10)

,repo_id,created_at
0,1,2020-10-01 11:55:32
1,27,2020-11-12 05:53:20
2,28,2017-11-11 06:42:24
3,35,2016-01-01 12:57:28
4,43,2017-03-28 08:48:29
5,68,2021-09-01 11:35:51
6,130,2021-12-24 14:39:18
7,131,2021-02-24 09:27:52
8,142,2020-07-01 00:44:55
9,144,2020-10-15 18:38:07


In [53]:
TEMP_TABLE_NAME15 = 'hx_repo_actor_birank_pull_repo_newest_repo_name'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        repo_id     BIGINT,
        repo_name    STRING
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME15
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520023554807g6v288ms2>

In [54]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME3}
        SELECT
            {TEMP_TABLE_NAME2}.repo_id,
            repo_name
        FROM {TEMP_TABLE_NAME1}, {TEMP_TABLE_NAME2}
        WHERE {TEMP_TABLE_NAME1}.repo_id = {TEMP_TABLE_NAME2}.repo_id AND {TEMP_TABLE_NAME1}.created_at = {TEMP_TABLE_NAME2}.created_at
        ORDER BY repo_id'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME15_2,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME15_1,
    TEMP_TABLE_NAME3=TEMP_TABLE_NAME15
)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [55]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME15))
data.head(10)

,repo_id,repo_name
0,1,mojombo/grit
1,27,rubinius/rubinius
2,28,mojombo/god
3,35,defunkt/exception_logger
4,43,technoweenie/attachment_fu
5,68,macournoyer/thin
6,130,ruby-git/ruby-git
7,131,ezmobius/bmhsearch
8,142,abhay/calais
9,144,mojombo/chronic


actor:id---name

In [56]:
TEMP_TABLE_NAME16_1 = 'hx_repo_actor_pull_request_actor_withdate'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        actor_id     BIGINT,
        actor_name    STRING,
        created_at    DATETIME
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME16_1
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520023620319goh7k963>

In [57]:
actor_name_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME}
        SELECT
            actor_id,
            actor_login,
            created_at
        FROM ods_github_log
        WHERE (type = 'PullRequestEvent') AND (action = 'opened')
        AND pt>='20160101'
        AND pt<'20220101'
        AND actor_id IN (
            SELECT actor_id
            FROM ods_github_log
            WHERE (type = 'PullRequestEvent') AND (action = 'opened')
            AND pt>='20160101'
            AND pt<'20220101'
            GROUP BY actor_id
            HAVING COUNT(*) > 10
        )
        ORDER BY actor_id
'''.format(TEMP_TABLE_NAME=TEMP_TABLE_NAME16_1)
table = o.execute_sql(actor_name_sql, hints={"odps.sql.submit.mode" : "script"})

In [58]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME16_1))
data.head(10)

,actor_id,actor_name,created_at
0,1,mojombo,2021-11-05 21:26:21
1,1,mojombo,2020-02-12 01:06:25
2,1,mojombo,2018-03-23 05:43:24
3,1,mojombo,2020-08-25 05:41:10
4,1,mojombo,2021-11-05 14:49:48
5,1,mojombo,2016-05-14 14:00:50
6,1,mojombo,2018-10-19 08:06:24
7,1,mojombo,2020-08-28 05:51:46
8,1,mojombo,2017-11-28 14:31:18
9,1,mojombo,2020-08-01 13:13:20


In [59]:
TEMP_TABLE_NAME16_2 = 'hx_repo_actor_pull_request_actor_maxdate'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        actor_id     BIGINT,
        max_created_at    DATETIME
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME16_2
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520024136101guaizjm7>

In [60]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME2}
        SELECT
            actor_id,
            MAX(created_at)
        FROM {TEMP_TABLE_NAME1}
        GROUP BY actor_id
        ORDER BY actor_id'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME16_1,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME16_2)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [61]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME16_2))
data.head(10)

,actor_id,max_created_at
0,1,2021-11-10 20:42:36
1,4,2021-12-24 16:49:02
2,7,2021-11-30 11:24:43
3,17,2021-12-29 17:18:32
4,18,2021-11-30 17:08:37
5,19,2020-03-05 09:05:54
6,20,2021-12-26 21:46:04
7,21,2021-07-04 07:05:05
8,22,2021-09-17 15:26:29
9,25,2021-01-08 11:48:43


In [62]:
TEMP_TABLE_NAME16 = 'hx_repo_actor_birank_pull_repo_newest_actor_name'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        actor_id     BIGINT,
        actor_name    STRING
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME16
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520024145813gvcyk4pr2>

In [63]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_TABLE_NAME3}
        SELECT DISTINCT
            {TEMP_TABLE_NAME2}.actor_id,
            actor_name
        FROM {TEMP_TABLE_NAME1}, {TEMP_TABLE_NAME2}
        WHERE {TEMP_TABLE_NAME1}.actor_id = {TEMP_TABLE_NAME2}.actor_id AND {TEMP_TABLE_NAME1}.max_created_at = {TEMP_TABLE_NAME2}.created_at
        ORDER BY actor_id'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME16_2,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME16_1,
    TEMP_TABLE_NAME3=TEMP_TABLE_NAME16
)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [64]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME16))
data.head(10)

,actor_id,actor_name
0,1,mojombo
1,4,wycats
2,7,evanphx
3,17,vanpelt
4,18,wayneeseguin
5,19,brynary
6,20,kevinclark
7,21,technoweenie
8,22,macournoyer
9,25,caged


TEMP_TABLE_NAME8存放了repo_id---new_repo_id

TEMP_TABLE_NAME10存放了actor_id---new_actor_id

TEMP_TABLE_NAME15存放了repo_id---repo_name

TEMP_TABLE_NAME16存放了actor_id---actor_name

TEMP_TABLE_NAME17将存放new_repo_id---repo_name

TEMP_TABLE_NAME18将存放new_actor_id---actor_login

In [65]:
TEMP_TABLE_NAME17 = 'hx_repo_actor_pull_request_new_repoid_name'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        new_repo_id     BIGINT,
        repo_name    STRING
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME17
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520024201741ghwysa2a>

In [66]:
repo_newid_name_sql = '''set odps.sql.validate.orderby.limit=false;
    INSERT INTO {TEMP_TABLE_NAME3}
    SELECT DISTINCT new_repo_id, repo_name
    FROM {TEMP_TABLE_NAME1}, {TEMP_TABLE_NAME2}
    WHERE {TEMP_TABLE_NAME1}.repo_id = {TEMP_TABLE_NAME2}.repo_id
    ORDER BY new_repo_id
'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME8,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME15,
    TEMP_TABLE_NAME3=TEMP_TABLE_NAME17
)
table = o.execute_sql(repo_newid_name_sql, hints={"odps.sql.submit.mode" : "script"})

In [67]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME17))
data.head(10)

,new_repo_id,repo_name
0,0,mojombo/grit
1,1,rubinius/rubinius
2,2,mojombo/god
3,3,defunkt/exception_logger
4,4,technoweenie/attachment_fu
5,5,macournoyer/thin
6,6,ruby-git/ruby-git
7,7,ezmobius/bmhsearch
8,8,abhay/calais
9,9,mojombo/chronic


In [68]:
pd_repo_newid_name = data.to_pandas()
pd_repo_newid_name.head()

,new_repo_id,repo_name
0,0,mojombo/grit
1,1,rubinius/rubinius
2,2,mojombo/god
3,3,defunkt/exception_logger
4,4,technoweenie/attachment_fu


In [69]:
TEMP_TABLE_NAME18 = 'hx_repo_actor_pull_request_new_actorid_name'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        new_actor_id     BIGINT,
        actor_name    STRING
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME18
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520024448283g4ned9o7>

In [70]:
actor_newid_name_sql = '''set odps.sql.validate.orderby.limit=false;
    INSERT INTO {TEMP_TABLE_NAME3}
    SELECT DISTINCT new_actor_id, actor_name
    FROM {TEMP_TABLE_NAME1}, {TEMP_TABLE_NAME2}
    WHERE {TEMP_TABLE_NAME1}.actor_id = {TEMP_TABLE_NAME2}.actor_id
    ORDER BY new_actor_id
'''.format(
    TEMP_TABLE_NAME1=TEMP_TABLE_NAME10,
    TEMP_TABLE_NAME2=TEMP_TABLE_NAME16,
    TEMP_TABLE_NAME3=TEMP_TABLE_NAME18
)
table = o.execute_sql(actor_newid_name_sql, hints={"odps.sql.submit.mode" : "script"})

In [71]:
data = DataFrame(o.get_table(TEMP_TABLE_NAME18))
data.head(10)

,new_actor_id,actor_name
0,0,mojombo
1,1,wycats
2,2,evanphx
3,3,vanpelt
4,4,wayneeseguin
5,5,brynary
6,6,kevinclark
7,7,technoweenie
8,8,macournoyer
9,9,caged


In [72]:
pd_actor_newid_name = data.to_pandas()
pd_actor_newid_name.head()

,new_actor_id,actor_name
0,0,mojombo
1,1,wycats
2,2,evanphx
3,3,vanpelt
4,4,wayneeseguin


pd_repo_newid_name存放了new_repo_id---repo_name的pd.df

pd_actor_newid_name存放了new_actor_id---actor_name的pd.df

### 以下为构建的burstbirank算法部分

In [73]:
import numpy as np
import scipy
import scipy.sparse as spa
# 为了不破坏new_pd_data原始数据，这里用test_data作为new_pd_data的副本加入运算
test_data = new_pd_data.copy()
row = test_data['new_actor_id'].values
col = test_data['new_repo_id'].values
# data = test_data['weight'].values
data=[1]*len(test_data['weight'])
# Weight矩阵是大小为Actors*Repos的稀疏矩阵,没有对应weight值的地方全为0
Weight = spa.coo_matrix((data,(row,col)))
# print(Weight.min())
# print(Weight.max())
# print(type(Weight))

In [74]:
# Degree_a、Degree_r分别代表Actors和Repos对应的的度列表
Degree_a = actor_degree_pd_data['actor_degree'].values
print(len(Degree_a))
print(type(Degree_a))
Degree_r = repo_degree_pd_data['repo_degree'].values
print(Degree_r)

1347313
<class 'numpy.ndarray'>
[ 1 33  6 ...  1  1  1]


In [75]:
def burstbirank(W,Da,Dr,alpha=0.85,beta=0.85,max_iter=200,tol=1.0e-4,verbose=False):
    '''
    输入:
        W::scipy's sparse matrix:Actors*Repos二分图网络的邻接矩阵
        Da::list:Actors的度列表
        Dr::list:Repos的度列表
        alpha, beta::float:超参数
        max_iter::int:最大迭代次数
        tol::float:检查收敛性的容错度
        verbose::boolean:运行时是否打印迭代信息
    输出:
        a, r::numpy.ndarray:Actor和Repos的burstbirank的值
    '''
    # W为A*R的矩阵,WT为R*A
    W = W.astype('float', copy=False)
    WT=W.T
    # Da_bi代表对角线元素为actor度平方根的倒数的对角矩阵,Dr_bi代表对角线元素为repo度平方根的倒数的对角矩阵
    # Sr代表Dr_bi*WT*Da_bi,Sa代表Da_bi*W*Dr_bi
    
    Da_bi=spa.diags(1/scipy.sqrt(Da))
    Dr_bi=spa.diags(1/scipy.sqrt(Dr))
    # print(Dr_bi.shape)
    # print(WT.shape)
    # print(Da_bi.shape)
    Sr=Dr_bi.dot(WT).dot(Da_bi)
    Sa=Sr.T
    # 随机初始化a0,r0
    a0 = np.repeat(1/len(Da),len(Da))
    r0 = np.repeat(1/len(Dr),len(Dr))
    a_last=a0.copy()
    r_last=r0.copy()
    end=0
    for i in range(max_iter):
        r = alpha * (Sr.dot(a_last)) + (1-alpha) * r0
        a = beta * (Sa.dot(r_last)) + (1-beta) * a0
        err_r=np.absolute(r-r_last).sum()
        err_a=np.absolute(a-a_last).sum()
        if verbose:
            print(
                "Iteration : {}; actor error: {}; repo error: {}".format(i, err_a, err_r)
            )
        if err_r < tol and err_a < tol:
            break
        a_last=a
        r_last=r
        end=i
    print(end)
    return a,r

In [76]:
actor_value,repo_value=burstbirank(Weight,Degree_a,Degree_r)

C:\Users\hp\AppData\Local\Temp/ipykernel_10792/2916817495.py:20: DeprecationWarning: scipy.sqrt is deprecated and will be removed in SciPy 2.0.0, use numpy.lib.scimath.sqrt instead
  Da_bi=spa.diags(1/scipy.sqrt(Da))
C:\Users\hp\AppData\Local\Temp/ipykernel_10792/2916817495.py:21: DeprecationWarning: scipy.sqrt is deprecated and will be removed in SciPy 2.0.0, use numpy.lib.scimath.sqrt instead
  Dr_bi=spa.diags(1/scipy.sqrt(Dr))


57


In [77]:
# 先看看最大的
print(np.argmax(repo_value))  #横着比较，返回列号
print(np.argmax(actor_value))  #横着比较，返回列号

1026966
414972


In [78]:
repo_value_list = repo_value.tolist()
repo = {'repo_value': repo_value_list}
repo_info = pd.DataFrame(repo)
repo_info.head(5)

,repo_value
0,1.268755e-07
1,5.554472e-07
2,2.652210e-07
3,1.096272e-07
4,9.181478e-08


In [79]:
repo_info.sort_values(by="repo_value" , inplace=True, ascending=False) 
repo_info['new_repo_id'] = repo_info.index
repo_info = repo_info.reset_index(drop=True)
repo_info['rank'] = repo_info.index+1
repo_info.head(10)


,repo_value,new_repo_id,rank
0,0.000014,1026966,1
1,0.000012,139313,2
2,0.000011,59756,3
3,0.000008,14527,4
4,0.000007,7874763,5
5,0.000007,600362,6
6,0.000007,1137269,7
7,0.000007,2001869,8
8,0.000007,38099,9
9,0.000007,252316,10


In [80]:
actor_value_list = actor_value.tolist()
actor = {'actor_value': actor_value_list}
actor_info = pd.DataFrame(actor)
actor_info.sort_values(by="actor_value" , inplace=True, ascending=False) 
actor_info['new_actor_id'] = actor_info.index
actor_info = actor_info.reset_index(drop=True)
actor_info['rank'] = actor_info.index+1
actor_info.head(10)


,actor_value,new_actor_id,rank
0,0.000067,414972,1
1,0.000012,673689,2
2,0.000011,361520,3
3,0.000010,334816,4
4,0.000009,1142911,5
5,0.000006,595188,6
6,0.000006,1014261,7
7,0.000006,571640,8
8,0.000006,1065468,9
9,0.000006,315487,10


In [81]:
repo_result = pd.merge(repo_info, pd_repo_newid_name, on='new_repo_id')
order = ['new_repo_id', 'repo_name', 'repo_value','rank']
repo_result = repo_result[order]
repo_result.head(10)

,new_repo_id,repo_name,repo_value,rank
0,1026966,firstcontributions/first-contributions,0.000014,1
1,139313,jlord/patchwork,0.000012,2
2,59756,DefinitelyTyped/DefinitelyTyped,0.000011,3
3,14527,octocat/Spoon-Knife,0.000008,4
4,7874763,education/GitHubGraduation-2021,0.000007,5
5,600362,Homebrew/homebrew-core,0.000007,6
6,1137269,MicrosoftDocs/azure-docs,0.000007,7
7,2001869,AliceWonderland/hacktoberfest,0.000007,8
8,38099,Homebrew/homebrew-cask,0.000007,9
9,252316,freeCodeCamp/freeCodeCamp,0.000007,10


In [82]:
actor_result = pd.merge(actor_info, pd_actor_newid_name, on='new_actor_id')
order = ['new_actor_id', 'actor_name', 'actor_value','rank']
actor_result = actor_result[order]
actor_result.head(20)

,new_actor_id,actor_name,actor_value,rank
0,414972,direwolf-github,0.000067,1
1,673689,codacy-badger,0.000012,2
2,361520,gitter-badger,0.000011,3
3,334816,xmo-odoo,0.000010,4
4,1142911,codeserver-test1,0.000009,5
5,595188,QualitySoftwareDeveloper,0.000006,6
6,1014261,git2e,0.000006,7
7,571640,idsb3t1,0.000006,8
8,1065468,codeserver-service-qa,0.000006,9
9,315487,waffle-iron,0.000006,10


## 统计信息

### watch

#### 查看ods_github_log表发现有名为WatchEvent的type，可以从中得到watch相关信息

In [83]:
TEMP_WATCH_TABLE00 = 'hx_repo_actor_birank_type'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_WATCH_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_WATCH_TABLE}
    (
        type    STRING
    );
'''.format(
    TEMP_WATCH_TABLE=TEMP_WATCH_TABLE00
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520024556996gixb2i5s2>

In [84]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_WATCH_TABLE}
        SELECT DISTINCT type
        FROM ods_github_log
        WHERE actor_login NOT LIKE '%bot%'
        AND pt>='20160101' AND pt<'20220101'
        '''.format(
    TEMP_WATCH_TABLE=TEMP_WATCH_TABLE00)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [85]:
data = DataFrame(o.get_table(TEMP_WATCH_TABLE00))
data.head()

,type
0,GollumEvent
1,PullRequestReviewCommentEvent
2,CommitCommentEvent
3,PublicEvent
4,IssueCommentEvent
5,ReleaseEvent
6,ForkEvent
7,PullRequestEvent
8,CreateEvent
9,MemberEvent


#### 统计各repo被watch的数目，建立 repo_id --- watch_cnt 对应关系表

In [86]:
TEMP_WATCH_TABLE1 = 'hx_repo_actor_birank_watch'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_WATCH_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_WATCH_TABLE}
    (
        repo_id     BIGINT,
        watch_cnt    BIGINT
    );
'''.format(
    TEMP_WATCH_TABLE=TEMP_WATCH_TABLE1
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520024911333g7kxsa2a>

In [87]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_WATCH_TABLE2}
        SELECT repo_id, count(*) AS watch_cnt
        FROM ods_github_log
        WHERE type="WatchEvent" AND repo_id IN (SELECT DISTINCT repo_id FROM {TEMP_WATCH_TABLE1})
        AND pt>='20160101' AND pt<'20220101'
        GROUP BY repo_id
        ORDER BY watch_cnt desc
        '''.format(
    TEMP_WATCH_TABLE1=TEMP_TABLE_NAME7,
    TEMP_WATCH_TABLE2=TEMP_WATCH_TABLE1)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [88]:
data = DataFrame(o.get_table(TEMP_WATCH_TABLE1))
data.head(10)

,repo_id,watch_cnt
0,177736533,367097
1,28457823,353140
2,60493101,222322
3,11730342,215106
4,85077558,196137
5,13491895,189269
6,54346799,181217
7,10270250,177594
8,45717250,171686
9,21737465,171290


#### 建立 repo_id --- new_repo_id 对应关系表

In [89]:
TEMP_WATCH_TABLE2_1 = 'hx_repo_actor_birank_pull_forwatch_only_repoid'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_WATCH_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_WATCH_TABLE}
    (
        repo_id     BIGINT
    );
'''.format(
    TEMP_WATCH_TABLE=TEMP_WATCH_TABLE2_1
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 2022052002512170gbb1igr>

In [90]:
get_repoid_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_WATCH_TABLE2}
        SELECT DISTINCT repo_id
        FROM {TEMP_WATCH_TABLE1}
        ORDER BY repo_id
'''.format(
    TEMP_WATCH_TABLE1=TEMP_TABLE_NAME7,
    TEMP_WATCH_TABLE2=TEMP_WATCH_TABLE2_1
)
table = o.execute_sql(get_repoid_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [91]:
data = DataFrame(o.get_table(TEMP_WATCH_TABLE2_1))
data.head(15)

,repo_id
0,1
1,27
2,28
3,35
4,43
5,68
6,130
7,131
8,142
9,144


In [92]:
TEMP_WATCH_TABLE2 = 'hx_repo_actor_birank_pull_forwatch_new_repoid'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_WATCH_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_WATCH_TABLE}
    (
        repo_id     BIGINT,
        new_repo_id    BIGINT
    );
'''.format(
    TEMP_WATCH_TABLE=TEMP_WATCH_TABLE2
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520025127635gsh6uq2>

In [93]:
insert_new_repoid_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_WATCH_TABLE2}
        SELECT repo_id, row_number() over() - 1 as new_repo_id
        FROM {TEMP_WATCH_TABLE1}
        ORDER BY repo_id
'''.format(
    TEMP_WATCH_TABLE1=TEMP_WATCH_TABLE2_1,
    TEMP_WATCH_TABLE2=TEMP_WATCH_TABLE2
)
table = o.execute_sql(insert_new_repoid_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [94]:
data = DataFrame(o.get_table(TEMP_WATCH_TABLE2))
data.head(15)

,repo_id,new_repo_id
0,1,0
1,27,1
2,28,2
3,35,3
4,43,4
5,68,5
6,130,6
7,131,7
8,142,8
9,144,9


In [95]:
# 查看最大new_repo_id(用于保证与算法中的new_repo_id一致)
select_max_new_repo_id = '''SELECT MAX(new_repo_id) FROM {TEMP_WATCH_TABLE}'''.format(TEMP_WATCH_TABLE=TEMP_WATCH_TABLE2)
table_max_new_repo_id = o.execute_sql(select_max_new_repo_id, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})
with table_max_new_repo_id.open_reader(tunnel=True) as reader:
     for record in reader:
            print(record)

odps.Record {
  _c0   9021971
}


#### 建立new_repo_id --- watch_cnt对应关系表

In [96]:
TEMP_WATCH_TABLE4 = 'hx_repo_actor_birank_newid_watch_cnt'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_WATCH_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_WATCH_TABLE}
    (
        new_repo_id    BIGINT,
        watch_cnt    BIGINT DEFAULT 0
    );
'''.format(
    TEMP_WATCH_TABLE=TEMP_WATCH_TABLE4
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520025136883gtw6k963>

In [97]:
newid_watch_cnt_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_WATCH_TABLE3}
        SELECT DISTINCT new_repo_id, watch_cnt
        FROM {TEMP_WATCH_TABLE1}, {TEMP_WATCH_TABLE2}
        WHERE {TEMP_WATCH_TABLE1}.repo_id = {TEMP_WATCH_TABLE2}.repo_id
        ORDER BY watch_cnt desc
'''.format(
    TEMP_WATCH_TABLE1=TEMP_WATCH_TABLE1,
    TEMP_WATCH_TABLE2=TEMP_WATCH_TABLE2,
    TEMP_WATCH_TABLE3=TEMP_WATCH_TABLE4
)
table = o.execute_sql(newid_watch_cnt_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [98]:
data = DataFrame(o.get_table(TEMP_WATCH_TABLE4))
data.head(15)

,new_repo_id,watch_cnt
0,3718661,367097
1,252316,353140
2,790124,222322
3,105344,215106
4,1467892,196137
5,119738,189269
6,640979,181217
7,93501,177594
8,441142,171686
9,192504,171290


#### 建立new_repo_id --- watch_rank对应关系

In [99]:
TEMP_WATCH_TABLE5 = 'hx_repo_actor_birank_watch_rank'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_WATCH_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_WATCH_TABLE}
    (
        new_repo_id    BIGINT,
        watch_rank    BIGINT
    );
'''.format(
    TEMP_WATCH_TABLE=TEMP_WATCH_TABLE5
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520025144344g1vzsa2a>

In [100]:
insert_watch_rank_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_WATCH_TABLE2}
        SELECT new_repo_id, row_number() over() AS rank
        FROM {TEMP_WATCH_TABLE1}
        ORDER BY rank
'''.format(
    TEMP_WATCH_TABLE1=TEMP_WATCH_TABLE4,
    TEMP_WATCH_TABLE2=TEMP_WATCH_TABLE5
)
table = o.execute_sql(insert_watch_rank_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [101]:
data = DataFrame(o.get_table(TEMP_WATCH_TABLE5))
data.head(10)

,new_repo_id,watch_rank
0,3718661,1
1,252316,2
2,790124,3
3,105344,4
4,1467892,5
5,119738,6
6,640979,7
7,93501,8
8,441142,9
9,192504,10


In [102]:
pd_idwatch = data.to_pandas()
pd_idwatch.head()

,new_repo_id,watch_rank
0,3718661,1
1,252316,2
2,790124,3
3,105344,4
4,1467892,5


#### 为方便展示，建立new_repo_id---repo_name---watch_rank对应关系表

In [103]:
TEMP_WATCH_TABLE6 = 'hx_repo_actor_birank_pull_watch_repo_withname_rank'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_WATCH_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_WATCH_TABLE}
    (
        new_repo_id     BIGINT,
        repo_name    STRING,
        watch_rank    BIGINT
    );
'''.format(
    TEMP_WATCH_TABLE=TEMP_WATCH_TABLE6
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520025222653gyf0ta2a>

In [104]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_WATCH_TABLE4}
        SELECT
            {TEMP_WATCH_TABLE1}.new_repo_id,
            repo_name,
            watch_rank
        FROM {TEMP_WATCH_TABLE1}, {TEMP_WATCH_TABLE2}, {TEMP_WATCH_TABLE3}
        WHERE {TEMP_WATCH_TABLE1}.new_repo_id = {TEMP_WATCH_TABLE3}.new_repo_id
        AND {TEMP_WATCH_TABLE3}.repo_id = {TEMP_WATCH_TABLE2}.repo_id
        ORDER BY watch_rank'''.format(
    TEMP_WATCH_TABLE1=TEMP_WATCH_TABLE5,
    TEMP_WATCH_TABLE2=TEMP_TABLE_NAME15,
    TEMP_WATCH_TABLE3=TEMP_WATCH_TABLE2,
    TEMP_WATCH_TABLE4=TEMP_WATCH_TABLE6
)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [105]:
data = DataFrame(o.get_table(TEMP_WATCH_TABLE6))
data.head(10)

,new_repo_id,repo_name,watch_rank
0,3718661,996icu/996.ICU,1
1,252316,freeCodeCamp/freeCodeCamp,2
2,790124,jwasham/coding-interview-university,3
3,105344,vuejs/vue,4
4,1467892,kamranahmedse/developer-roadmap,5
5,119738,EbookFoundation/free-programming-books,6
6,640979,public-apis/public-apis,7
7,93501,facebook/react,8
8,441142,tensorflow/tensorflow,9
9,192504,sindresorhus/awesome,10


### followers

#### 从TEMP_TABLE_NAME16_1中取出actor_id---actor_name对应表

In [106]:
TEMP_FOLLOW_TABLE0 = 'hx_repo_actor_birank_pull_forfollower_actorid'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_FOLLOW_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_FOLLOW_TABLE}
    (
        actor_id     BIGINT,
        actor_name    STRING
    );
'''.format(
    TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE0
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520025236714gi1205pr2>

In [107]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_FOLLOW_TABLE2}
        SELECT DISTINCT actor_id, actor_name
        FROM {TEMP_FOLLOW_TABLE1}
        ORDER BY actor_id'''.format(
    TEMP_FOLLOW_TABLE1=TEMP_TABLE_NAME16_1,
    TEMP_FOLLOW_TABLE2=TEMP_FOLLOW_TABLE0)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [108]:
data = DataFrame(o.get_table(TEMP_FOLLOW_TABLE0))
data.head(10)

,actor_id,actor_name
0,1,mojombo
1,4,wycats
2,7,evanphx
3,17,vanpelt
4,18,wayneeseguin
5,19,brynary
6,20,kevinclark
7,21,technoweenie
8,22,macournoyer
9,25,Caged


In [109]:
# 39191的name
select_39191 = '''SELECT * FROM {TEMP_FOLLOW_TABLE} WHERE actor_id = 39191'''.format(TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE0)
table_39191 = o.execute_sql(select_39191, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})
with table_39191.open_reader(tunnel=True) as reader:
    for record in reader:
        print(record)

odps.Record {
  actor_id            39191
  actor_name          'paulirish'
}


#### 从ods_github_users中获得actor_id---actor_name唯一对应表

In [110]:
TEMP_FOLLOW_TABLE1 = 'hx_repo_actor_birank_pull_actorid'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_FOLLOW_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_FOLLOW_TABLE}
    (
        actor_name    STRING,
        follower_cnt    BIGINT DEFAULT 0,
        time    DATE
    );
'''.format(
    TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE1
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520025248140g6p2igr>

In [111]:
follower_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_FOLLOW_TABLE1}
        SELECT login, CHAR_MATCHCOUNT(followers, ',') + 1 AS follower_cnt, lastupdatedat
        FROM ods_github_users
        WHERE login NOT LIKE '%bot%' AND TO_CHAR(lastupdatedat,'yyyymmdd') >= '20160101'
        AND login in (SELECT actor_name FROM {TEMP_FOLLOW_TABLE0})
        ORDER BY follower_cnt desc
        '''.format(
    TEMP_FOLLOW_TABLE0=TEMP_FOLLOW_TABLE0,
    TEMP_FOLLOW_TABLE1=TEMP_FOLLOW_TABLE1
)
table = o.execute_sql(follower_sql, hints={"odps.sql.submit.mode" : "script"})

In [112]:
data = DataFrame(o.get_table(TEMP_FOLLOW_TABLE1))
data.head(10)

,actor_name,follower_cnt,time
0,torvalds,148982,2022-01-02
1,torvalds,146716,2022-01-02
2,yyx990803,77598,2022-01-24
3,yyx990803,76771,2022-01-24
4,yyx990803,75801,2022-01-24
5,gaearon,71341,2022-01-13
6,gaearon,70503,2022-01-13
7,gaearon,69646,2022-01-13
8,JakeWharton,61869,2022-01-22
9,JakeWharton,61680,2022-01-22


#### 将actor_name以actor_id代替，得到actor_id---follower_cnt---time对应表

In [113]:
TEMP_FOLLOW_TABLE1_00 = 'hx_repo_actor_birank_pull_id_to_name'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_FOLLOW_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_FOLLOW_TABLE}
    (
        actor_id    BIGINT,
        follower_cnt    BIGINT,
        time    DATE
    );
'''.format(
    TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE1_00
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520025303230gcs5n4pr2>

In [114]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_FOLLOW_TABLE2}
        SELECT actor_id, follower_cnt, time
        FROM {TEMP_FOLLOW_TABLE0}, {TEMP_FOLLOW_TABLE1}
        WHERE {TEMP_FOLLOW_TABLE0}.actor_name = {TEMP_FOLLOW_TABLE1}.actor_name
        ORDER BY follower_cnt DESC
        '''.format(
    TEMP_FOLLOW_TABLE0=TEMP_FOLLOW_TABLE0,
    TEMP_FOLLOW_TABLE1=TEMP_FOLLOW_TABLE1,
    TEMP_FOLLOW_TABLE2=TEMP_FOLLOW_TABLE1_00
)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [115]:
data = DataFrame(o.get_table(TEMP_FOLLOW_TABLE1_00))
data.head(10)

,actor_id,follower_cnt,time
0,1024025,148982,2022-01-02
1,1024025,146716,2022-01-02
2,499550,77598,2022-01-24
3,499550,76771,2022-01-24
4,499550,75801,2022-01-24
5,810438,71341,2022-01-13
6,810438,70503,2022-01-13
7,810438,69646,2022-01-13
8,66577,61869,2022-01-22
9,66577,61680,2022-01-22


#### 获得最新的actor_id对应的follow_cnt

In [116]:
TEMP_FOLLOW_TABLE1_0 = 'hx_repo_actor_birank_pull_today_newest'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_FOLLOW_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_FOLLOW_TABLE}
    (
        actor_id    BIGINT,
        follow_cnt    BIGINT,
        time    DATE
    );
'''.format(
    TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE1_0
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520025310688gi5rdgr>

In [117]:
today_newest_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_FOLLOW_TABLE1}
        SELECT actor_id, MAX(follower_cnt) AS follow_cnt, time
        FROM {TEMP_FOLLOW_TABLE0}
        GROUP BY actor_id, time
        ORDER BY follow_cnt DESC
        '''.format(
    TEMP_FOLLOW_TABLE0=TEMP_FOLLOW_TABLE1_00,
    TEMP_FOLLOW_TABLE1=TEMP_FOLLOW_TABLE1_0
)
table = o.execute_sql(today_newest_sql, hints={"odps.sql.submit.mode" : "script"})

In [118]:
data = DataFrame(o.get_table(TEMP_FOLLOW_TABLE1_0))
data.head(10)

,actor_id,follow_cnt,time
0,1024025,148982,2022-01-02
1,499550,77598,2022-01-24
2,810438,71341,2022-01-13
3,66577,61869,2022-01-22
4,170270,46660,2022-01-29
5,150330,33945,2022-01-28
6,39191,30116,2021-12-27
7,8547538,27087,2022-01-17
8,176013,26789,2022-01-26
9,80,26315,2022-01-21


#### 仅保留actor_id---follow_cnt信息

In [119]:
TEMP_FOLLOW_TABLE2 = 'hx_repo_actor_birank_pull_latest_time'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_FOLLOW_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_FOLLOW_TABLE}
    (
        actor_id    BIGINT,
        follow_cnt    BIGINT DEFAULT 0
    );
'''.format(
    TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE2
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 2022052002531551gzcxfgr>

In [120]:
latest_time_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_FOLLOW_TABLE1}
        SELECT actor_id, follow_cnt
        FROM {TEMP_FOLLOW_TABLE0}
        ORDER BY follow_cnt DESC
        '''.format(
    TEMP_FOLLOW_TABLE0=TEMP_FOLLOW_TABLE1_0,
    TEMP_FOLLOW_TABLE1=TEMP_FOLLOW_TABLE2
)
table = o.execute_sql(latest_time_sql, hints={"odps.sql.submit.mode" : "script"})

In [121]:
data = DataFrame(o.get_table(TEMP_FOLLOW_TABLE2))
data.head(10)

,actor_id,follow_cnt
0,1024025,148982
1,499550,77598
2,810438,71341
3,66577,61869
4,170270,46660
5,150330,33945
6,39191,30116
7,8547538,27087
8,176013,26789
9,80,26315


#### 建立actor_id---new_actor_id唯一对应关系

In [122]:
TEMP_FOLLOW_TABLE3_0 = 'hx_repo_actor_birank_only_actor_id'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_FOLLOW_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_FOLLOW_TABLE}
    (
        actor_id    BIGINT
    );
'''.format(
    TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE3_0
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520025320516g2ia2i5s2>

In [123]:
select_only_actorid_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_FOLLOW_TABLE2}
        SELECT DISTINCT actor_id
        FROM {TEMP_FOLLOW_TABLE1}
        ORDER BY actor_id
'''.format(
    TEMP_FOLLOW_TABLE1=TEMP_FOLLOW_TABLE0,
    TEMP_FOLLOW_TABLE2=TEMP_FOLLOW_TABLE3_0
)
table = o.execute_sql(select_only_actorid_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [124]:
data = DataFrame(o.get_table(TEMP_FOLLOW_TABLE3_0))
data.head(10)

,actor_id
0,1
1,4
2,7
3,17
4,18
5,19
6,20
7,21
8,22
9,25


In [125]:
TEMP_FOLLOW_TABLE3 = 'hx_repo_actor_birank_new_id'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_FOLLOW_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_FOLLOW_TABLE}
    (
        actor_id    BIGINT,
        new_actor_id    BIGINT
    );
'''.format(
    TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE3
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 2022052002532561gdycj8o7>

In [126]:
insert_new_actorid_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_FOLLOW_TABLE2}
        SELECT actor_id, row_number() over() - 1 as new_actor_id
        FROM {TEMP_FOLLOW_TABLE1}
        ORDER BY actor_id
'''.format(
    TEMP_FOLLOW_TABLE1=TEMP_FOLLOW_TABLE3_0,
    TEMP_FOLLOW_TABLE2=TEMP_FOLLOW_TABLE3
)
table = o.execute_sql(insert_new_actorid_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [127]:
data = DataFrame(o.get_table(TEMP_FOLLOW_TABLE3))
data.head(10)

,actor_id,new_actor_id
0,1,0
1,4,1
2,7,2
3,17,3
4,18,4
5,19,5
6,20,6
7,21,7
8,22,8
9,25,9


In [128]:
# 查看actor_id=39191对应的new_actor_id
select_39191 = '''SELECT * FROM {TEMP_FOLLOW_TABLE} WHERE actor_id = 39191'''.format(TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE3)
table_39191 = o.execute_sql(select_39191, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})
with table_39191.open_reader(tunnel=True) as reader:
    for record in reader:
        print(record)

odps.Record {
  actor_id                39191
  new_actor_id            8577
}


In [129]:
# 查看最大new_actor_id(用于保证与算法中的new_actor_id一致)
select_max_new_actor_id = '''SELECT MAX(new_actor_id) FROM {TEMP_FOLLOW_TABLE}'''.format(TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE3)
table_max_new_actor_id = o.execute_sql(select_max_new_actor_id, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})
with table_max_new_actor_id.open_reader(tunnel=True) as reader:
     for record in reader:
            print(record)

odps.Record {
  _c0   1350641
}


#### 建立new_actor_id---follower_cnt对应关系

In [130]:
TEMP_FOLLOW_TABLE4 = 'hx_repo_actor_birank_follower_table_new'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_FOLLOW_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_FOLLOW_TABLE}
    (
        new_actor_id    BIGINT,
        follower_cnt    BIGINT DEFAULT 0
    );
'''.format(
    TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE4
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520025333781gj9lzjm7>

In [131]:
newid_follow_cnt_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_FOLLOW_TABLE3}
        SELECT new_actor_id, follow_cnt
        FROM {TEMP_FOLLOW_TABLE1}, {TEMP_FOLLOW_TABLE2}
        WHERE {TEMP_FOLLOW_TABLE1}.actor_id = {TEMP_FOLLOW_TABLE2}.actor_id
        ORDER BY follow_cnt desc
'''.format(
    TEMP_FOLLOW_TABLE1=TEMP_FOLLOW_TABLE2,
    TEMP_FOLLOW_TABLE2=TEMP_FOLLOW_TABLE3,
    TEMP_FOLLOW_TABLE3=TEMP_FOLLOW_TABLE4
)
table = o.execute_sql(newid_follow_cnt_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [132]:
data = DataFrame(o.get_table(TEMP_FOLLOW_TABLE4))
data.head(10)

,new_actor_id,follower_cnt
0,101568,148982
1,60176,77598
2,85511,71341
3,13417,61869
4,28245,46660
5,25627,33945
6,8577,30116
7,362935,27087
8,28962,26789
9,26,26315


#### 得到new_actor_id---follower_rank对应关系

In [133]:
TEMP_FOLLOW_TABLE5 = 'hx_repo_actor_birank_follower_rank'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_FOLLOW_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_FOLLOW_TABLE}
    (
        new_actor_id    BIGINT,
        follower_rank    BIGINT
    );
'''.format(
    TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE5
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 2022052002533890gdo8k963>

In [134]:
insert_rank_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_FOLLOW_TABLE2}
        SELECT new_actor_id, row_number() over() AS rank
        FROM {TEMP_FOLLOW_TABLE1}
        ORDER BY rank
'''.format(
    TEMP_FOLLOW_TABLE1=TEMP_FOLLOW_TABLE4,
    TEMP_FOLLOW_TABLE2=TEMP_FOLLOW_TABLE5
)
table = o.execute_sql(insert_rank_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [135]:
data = DataFrame(o.get_table(TEMP_FOLLOW_TABLE5))
data.head(10)

,new_actor_id,follower_rank
0,101568,1
1,60176,2
2,85511,3
3,13417,4
4,28245,5
5,25627,6
6,8577,7
7,362935,8
8,28962,9
9,26,10


In [136]:
select_39191 = '''SELECT MAX(new_actor_id) FROM {TEMP_FOLLOW_TABLE}'''.format(TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE5)
table_39191 = o.execute_sql(select_39191, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})
with table_39191.open_reader(tunnel=True) as reader:
    for record in reader:
        print(record)

odps.Record {
  _c0   1350558
}


In [137]:
pd_idfollow = data.to_pandas()
pd_idfollow.head()

,new_actor_id,follower_rank
0,101568,1
1,60176,2
2,85511,3
3,13417,4
4,28245,5


#### 为便于展示，建立new_actor_id---actor_name---rank对应关系表

由于用户可能改名，actor_name可能会改变，但actor_id是不变的。所以，选取时间最近的actor_id对应的actor_name作为唯一对应的用户名，以便在github查询该用户名时能够得到相应页面

In [138]:
TEMP_FOLLOW_TABLE7 = 'hx_repo_actor_birank_pull_follower_actor_newid_and_name'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_FOLLOW_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_FOLLOW_TABLE}
    (
        new_actor_id     BIGINT,
        actor_name    STRING,
        follower_rank    BIGINT
    );
'''.format(
    TEMP_FOLLOW_TABLE=TEMP_FOLLOW_TABLE7
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520025346469gi7264pr2>

In [139]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_FOLLOW_TABLE4}
        SELECT
            {TEMP_FOLLOW_TABLE1}.new_actor_id,
            actor_name,
            follower_rank
        FROM {TEMP_FOLLOW_TABLE1}, {TEMP_FOLLOW_TABLE2}, {TEMP_FOLLOW_TABLE3}
        WHERE {TEMP_FOLLOW_TABLE1}.new_actor_id = {TEMP_FOLLOW_TABLE3}.new_actor_id
        AND {TEMP_FOLLOW_TABLE3}.actor_id = {TEMP_FOLLOW_TABLE2}.actor_id
        ORDER BY follower_rank'''.format(
    TEMP_FOLLOW_TABLE1=TEMP_FOLLOW_TABLE5,
    TEMP_FOLLOW_TABLE2=TEMP_TABLE_NAME16,
    TEMP_FOLLOW_TABLE3=TEMP_FOLLOW_TABLE3,
    TEMP_FOLLOW_TABLE4=TEMP_FOLLOW_TABLE7
)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [140]:
data = DataFrame(o.get_table(TEMP_FOLLOW_TABLE7))
data.head(10)

,new_actor_id,actor_name,follower_rank
0,101568,torvalds,1
1,60176,yyx990803,2
2,85511,gaearon,3
3,13417,JakeWharton,4
4,28245,sindresorhus,5
5,25627,getify,6
6,8577,paulirish,7
7,362935,buckyroberts,8
8,28962,wesbos,9
9,26,ry,10


## 评价

#### repo

##### top10

In [141]:
burstbirank_repo_top10=repo_info['new_repo_id'][:10].tolist()
print(burstbirank_repo_top10)

[1026966, 139313, 59756, 14527, 7874763, 600362, 1137269, 2001869, 38099, 252316]


In [142]:
TEMP_EVAL_TABLE1 = 'hx_busrtbirank_watch_repo_top10'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_EVAL_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_EVAL_TABLE}
    (
        new_repo_id     BIGINT,
        watch_rank    BIGINT
    );
'''.format(
    TEMP_EVAL_TABLE=TEMP_EVAL_TABLE1
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520025352942g5tmhv21>

In [143]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_EVAL_TABLE2}
        SELECT new_repo_id, watch_rank
        FROM {TEMP_EVAL_TABLE1}
        WHERE new_repo_id = {id1} OR new_repo_id = {id2} OR new_repo_id = {id3} OR new_repo_id = {id4} 
        OR new_repo_id = {id5} OR new_repo_id = {id6} OR new_repo_id = {id7} OR new_repo_id = {id8}
        OR new_repo_id = {id9} OR new_repo_id = {id10}
'''.format(
    TEMP_EVAL_TABLE1=TEMP_WATCH_TABLE6,
    TEMP_EVAL_TABLE2=TEMP_EVAL_TABLE1,
    id1=burstbirank_repo_top10[0],
    id2=burstbirank_repo_top10[1],
    id3=burstbirank_repo_top10[2],
    id4=burstbirank_repo_top10[3],
    id5=burstbirank_repo_top10[4],
    id6=burstbirank_repo_top10[5],
    id7=burstbirank_repo_top10[6],
    id8=burstbirank_repo_top10[7],
    id9=burstbirank_repo_top10[8],
    id10=burstbirank_repo_top10[9]
)
table = o.execute_sql(sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [144]:
data = DataFrame(o.get_table(TEMP_EVAL_TABLE1))
burstbirank_repo_top10_inwatch = data.to_pandas()
repo_result_top10 = pd.DataFrame(repo_result, columns=['new_repo_id','repo_name','rank'])[:10]
# 合并burstbirank_repo_top10_inwatch和repo_result_top10
burstbirank_watch_top10 = pd.merge(repo_result_top10, burstbirank_repo_top10_inwatch, on='new_repo_id')
burstbirank_watch_top10

,new_repo_id,repo_name,rank,watch_rank
0,1026966,firstcontributions/first-contributions,1,437
1,139313,jlord/patchwork,2,24375
2,59756,DefinitelyTyped/DefinitelyTyped,3,236
3,14527,octocat/Spoon-Knife,4,9653
4,7874763,education/GitHubGraduation-2021,5,18242
5,600362,Homebrew/homebrew-core,6,1646
6,1137269,MicrosoftDocs/azure-docs,7,3357
7,2001869,AliceWonderland/hacktoberfest,8,28425
8,38099,Homebrew/homebrew-cask,9,1491
9,252316,freeCodeCamp/freeCodeCamp,10,2


##### top20

In [145]:
burstbirank_repo_top20=repo_info['new_repo_id'][:20].tolist()
print(burstbirank_repo_top20)

[1026966, 139313, 59756, 14527, 7874763, 600362, 1137269, 2001869, 38099, 252316, 2288058, 38251, 326560, 561908, 456536, 181883, 46845, 441142, 71210, 96463]


In [146]:
TEMP_EVAL_TABLE2 = 'hx_busrtbirank_watch_repo_top20'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_EVAL_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_EVAL_TABLE}
    (
        new_repo_id     BIGINT,
        watch_rank    BIGINT
    );
'''.format(
    TEMP_EVAL_TABLE=TEMP_EVAL_TABLE2
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520025357713gy1h1ii5>

In [147]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_EVAL_TABLE2}
        SELECT new_repo_id, watch_rank
        FROM {TEMP_EVAL_TABLE1}
        WHERE new_repo_id = {id1} OR new_repo_id = {id2} OR new_repo_id = {id3} OR new_repo_id = {id4} 
        OR new_repo_id = {id5} OR new_repo_id = {id6} OR new_repo_id = {id7} OR new_repo_id = {id8}
        OR new_repo_id = {id9} OR new_repo_id = {id10} OR new_repo_id = {id11} OR new_repo_id = {id12}
        OR new_repo_id = {id13} OR new_repo_id = {id14} OR new_repo_id = {id15} OR new_repo_id = {id16}
        OR new_repo_id = {id17} OR new_repo_id = {id18} OR new_repo_id = {id19} OR new_repo_id = {id20}
'''.format(
    TEMP_EVAL_TABLE1=TEMP_WATCH_TABLE6,
    TEMP_EVAL_TABLE2=TEMP_EVAL_TABLE2,
    id1=burstbirank_repo_top20[0],
    id2=burstbirank_repo_top20[1],
    id3=burstbirank_repo_top20[2],
    id4=burstbirank_repo_top20[3],
    id5=burstbirank_repo_top20[4],
    id6=burstbirank_repo_top20[5],
    id7=burstbirank_repo_top20[6],
    id8=burstbirank_repo_top20[7],
    id9=burstbirank_repo_top20[8],
    id10=burstbirank_repo_top20[9],
    id11=burstbirank_repo_top20[10],
    id12=burstbirank_repo_top20[11],
    id13=burstbirank_repo_top20[12],
    id14=burstbirank_repo_top20[13],
    id15=burstbirank_repo_top20[14],
    id16=burstbirank_repo_top20[15],
    id17=burstbirank_repo_top20[16],
    id18=burstbirank_repo_top20[17],
    id19=burstbirank_repo_top20[18],
    id20=burstbirank_repo_top20[19]
)
table = o.execute_sql(sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [148]:
data = DataFrame(o.get_table(TEMP_EVAL_TABLE2))
burstbirank_repo_top20_inwatch = data.to_pandas()
repo_result_top20 = pd.DataFrame(repo_result, columns=['new_repo_id','repo_name','rank'])[:20]
# 合并burstbirank_repo_top20_inwatch和repo_result_top20
burstbirank_watch_top20 = pd.merge(repo_result_top20, burstbirank_repo_top20_inwatch, on='new_repo_id')
burstbirank_watch_top20

,new_repo_id,repo_name,rank,watch_rank
0,1026966,firstcontributions/first-contributions,1,437
1,139313,jlord/patchwork,2,24375
2,59756,DefinitelyTyped/DefinitelyTyped,3,236
3,14527,octocat/Spoon-Knife,4,9653
4,7874763,education/GitHubGraduation-2021,5,18242
5,600362,Homebrew/homebrew-core,6,1646
6,1137269,MicrosoftDocs/azure-docs,7,3357
7,2001869,AliceWonderland/hacktoberfest,8,28425
8,38099,Homebrew/homebrew-cask,9,1491
9,252316,freeCodeCamp/freeCodeCamp,10,2


##### top50

In [149]:
burstbirank_repo_top50=repo_info['new_repo_id'][:50].tolist()
print(burstbirank_repo_top50)

[1026966, 139313, 59756, 14527, 7874763, 600362, 1137269, 2001869, 38099, 252316, 2288058, 38251, 326560, 561908, 456536, 181883, 46845, 441142, 71210, 96463, 251278, 217461, 5031693, 114882, 7708185, 1981961, 139, 1672325, 241359, 760499, 940633, 2011602, 5922489, 1376566, 180406, 3096907, 204734, 257880, 8072, 1066192, 302700, 391536, 6499723, 392246, 9626, 217314, 513027, 240300, 246048, 357567]


In [150]:
TEMP_EVAL_TABLE3 = 'hx_busrtburstbirank_watch_repo_top50'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_EVAL_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_EVAL_TABLE}
    (
        new_repo_id     BIGINT,
        watch_rank    BIGINT
    );
'''.format(
    TEMP_EVAL_TABLE=TEMP_EVAL_TABLE3
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520025402411gpf264pr2>

In [151]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_EVAL_TABLE2}
        SELECT new_repo_id, watch_rank
        FROM {TEMP_EVAL_TABLE1}
        WHERE new_repo_id = {id1} OR new_repo_id = {id2} OR new_repo_id = {id3} OR new_repo_id = {id4} 
        OR new_repo_id = {id5} OR new_repo_id = {id6} OR new_repo_id = {id7} OR new_repo_id = {id8}
        OR new_repo_id = {id9} OR new_repo_id = {id10} OR new_repo_id = {id11} OR new_repo_id = {id12}
        OR new_repo_id = {id13} OR new_repo_id = {id14} OR new_repo_id = {id15} OR new_repo_id = {id16}
        OR new_repo_id = {id17} OR new_repo_id = {id18} OR new_repo_id = {id19} OR new_repo_id = {id20}
        OR new_repo_id = {id21} OR new_repo_id = {id22} OR new_repo_id = {id23} OR new_repo_id = {id24}
        OR new_repo_id = {id25} OR new_repo_id = {id26} OR new_repo_id = {id27} OR new_repo_id = {id28}
        OR new_repo_id = {id29} OR new_repo_id = {id30} OR new_repo_id = {id31} OR new_repo_id = {id32}
        OR new_repo_id = {id33} OR new_repo_id = {id34} OR new_repo_id = {id35} OR new_repo_id = {id36}
        OR new_repo_id = {id37} OR new_repo_id = {id38} OR new_repo_id = {id39} OR new_repo_id = {id40}
        OR new_repo_id = {id41} OR new_repo_id = {id42} OR new_repo_id = {id43} OR new_repo_id = {id44}
        OR new_repo_id = {id45} OR new_repo_id = {id46} OR new_repo_id = {id47} OR new_repo_id = {id48}
        OR new_repo_id = {id49} OR new_repo_id = {id50}
'''.format(
    TEMP_EVAL_TABLE1=TEMP_WATCH_TABLE6,
    TEMP_EVAL_TABLE2=TEMP_EVAL_TABLE3,
    id1=burstbirank_repo_top50[0], id2=burstbirank_repo_top50[1], id3=burstbirank_repo_top50[2], id4=burstbirank_repo_top50[3],
    id5=burstbirank_repo_top50[4], id6=burstbirank_repo_top50[5], id7=burstbirank_repo_top50[6], id8=burstbirank_repo_top50[7],
    id9=burstbirank_repo_top50[8], id10=burstbirank_repo_top50[9], id11=burstbirank_repo_top50[10], id12=burstbirank_repo_top50[11],
    id13=burstbirank_repo_top50[12], id14=burstbirank_repo_top50[13], id15=burstbirank_repo_top50[14], id16=burstbirank_repo_top50[15],
    id17=burstbirank_repo_top50[16], id18=burstbirank_repo_top50[17], id19=burstbirank_repo_top50[18], id20=burstbirank_repo_top50[19],
    id21=burstbirank_repo_top50[20], id22=burstbirank_repo_top50[21], id23=burstbirank_repo_top50[22], id24=burstbirank_repo_top50[23],
    id25=burstbirank_repo_top50[24], id26=burstbirank_repo_top50[25], id27=burstbirank_repo_top50[26], id28=burstbirank_repo_top50[27],
    id29=burstbirank_repo_top50[28], id30=burstbirank_repo_top50[29], id31=burstbirank_repo_top50[30], id32=burstbirank_repo_top50[31],
    id33=burstbirank_repo_top50[32], id34=burstbirank_repo_top50[33], id35=burstbirank_repo_top50[34], id36=burstbirank_repo_top50[35],
    id37=burstbirank_repo_top50[36], id38=burstbirank_repo_top50[37], id39=burstbirank_repo_top50[38], id40=burstbirank_repo_top50[39],
    id41=burstbirank_repo_top50[40], id42=burstbirank_repo_top50[41], id43=burstbirank_repo_top50[42], id44=burstbirank_repo_top50[43],
    id45=burstbirank_repo_top50[44], id46=burstbirank_repo_top50[45], id47=burstbirank_repo_top50[46], id48=burstbirank_repo_top50[47],
    id49=burstbirank_repo_top50[48], id50=burstbirank_repo_top50[49]
)
table = o.execute_sql(sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [152]:
data = DataFrame(o.get_table(TEMP_EVAL_TABLE3))
data.head(10)

,new_repo_id,watch_rank
0,252316,2
1,441142,9
2,392246,18
3,257880,36
4,1066192,38
5,181883,45
6,241359,47
7,204734,55
8,940633,78
9,8072,87


In [153]:
burstbirank_repo_top50_inwatch = data.to_pandas()
repo_result_top50 = pd.DataFrame(repo_result, columns=['new_repo_id','repo_name','rank'])[:50]
# 合并burstbirank_repo_top50_inwatch和repo_result_top50
burstbirank_watch_top50 = pd.merge(repo_result_top50, burstbirank_repo_top50_inwatch, on='new_repo_id')
burstbirank_watch_top50

,new_repo_id,repo_name,rank,watch_rank
0,1026966,firstcontributions/first-contributions,1,437
1,139313,jlord/patchwork,2,24375
2,59756,DefinitelyTyped/DefinitelyTyped,3,236
3,14527,octocat/Spoon-Knife,4,9653
4,7874763,education/GitHubGraduation-2021,5,18242
5,600362,Homebrew/homebrew-core,6,1646
6,1137269,MicrosoftDocs/azure-docs,7,3357
7,2001869,AliceWonderland/hacktoberfest,8,28425
8,38099,Homebrew/homebrew-cask,9,1491
9,252316,freeCodeCamp/freeCodeCamp,10,2


#### actor

In [154]:
burstbirank_actor_top10=actor_info['new_actor_id'][:10].tolist()
print(burstbirank_actor_top10)

[414972, 673689, 361520, 334816, 1142911, 595188, 1014261, 571640, 1065468, 315487]


In [155]:
TEMP_EVAL_TABLE4 = 'hx_burstbirank_follow_actor_top10'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_EVAL_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_EVAL_TABLE}
    (
        new_actor_id     BIGINT,
        follower_rank    BIGINT
    );
'''.format(
    TEMP_EVAL_TABLE=TEMP_EVAL_TABLE4
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520025408394gbm6w5pr2>

In [156]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_EVAL_TABLE2}
        SELECT new_actor_id, follower_rank
        FROM {TEMP_EVAL_TABLE1}
        WHERE new_actor_id = {id1} OR new_actor_id = {id2} OR new_actor_id = {id3} OR new_actor_id = {id4} 
        OR new_actor_id = {id5} OR new_actor_id = {id6} OR new_actor_id = {id7} OR new_actor_id = {id8}
        OR new_actor_id = {id9} OR new_actor_id = {id10}
'''.format(
    TEMP_EVAL_TABLE1=TEMP_FOLLOW_TABLE5,
    TEMP_EVAL_TABLE2=TEMP_EVAL_TABLE4,
    id1=burstbirank_actor_top10[0],
    id2=burstbirank_actor_top10[1],
    id3=burstbirank_actor_top10[2],
    id4=burstbirank_actor_top10[3],
    id5=burstbirank_actor_top10[4],
    id6=burstbirank_actor_top10[5],
    id7=burstbirank_actor_top10[6],
    id8=burstbirank_actor_top10[7],
    id9=burstbirank_actor_top10[8],
    id10=burstbirank_actor_top10[9]
)
table = o.execute_sql(sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [157]:
data = DataFrame(o.get_table(TEMP_EVAL_TABLE4))
data.head(10)

,new_actor_id,follower_rank
0,1142911,143070


In [158]:
burstbirank_actor_top10_infollow = data.to_pandas()
actor_result_top10 = pd.DataFrame(actor_result, columns=['new_actor_id','actor_name','rank'])[:10]
# 合并burstbirank_actor_top10_infollow和actor_result_top10
burstbirank_follow_top10 = pd.merge(actor_result_top10, burstbirank_actor_top10_infollow, on='new_actor_id')
burstbirank_follow_top10

,new_actor_id,actor_name,rank,follower_rank
0,1142911,codeserver-test1,5,143070


##### top20

In [159]:
burstbirank_actor_top20=actor_info['new_actor_id'][:20].tolist()
print(burstbirank_actor_top20)

[414972, 673689, 361520, 334816, 1142911, 595188, 1014261, 571640, 1065468, 315487, 122996, 221311, 1275949, 7479, 344388, 215104, 343522, 1235182, 1237106, 363095]


In [160]:
TEMP_EVAL_TABLE5 = 'hx_burstbirank_follow_actor_top20'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_EVAL_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_EVAL_TABLE}
    (
        new_actor_id     BIGINT,
        follower_rank    BIGINT
    );
'''.format(
    TEMP_EVAL_TABLE=TEMP_EVAL_TABLE5
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520025413160gmc7eom7>

In [161]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_EVAL_TABLE2}
        SELECT new_actor_id, follower_rank
        FROM {TEMP_EVAL_TABLE1}
        WHERE new_actor_id = {id1} OR new_actor_id = {id2} OR new_actor_id = {id3} OR new_actor_id = {id4} 
        OR new_actor_id = {id5} OR new_actor_id = {id6} OR new_actor_id = {id7} OR new_actor_id = {id8}
        OR new_actor_id = {id9} OR new_actor_id = {id10} OR new_actor_id = {id11} OR new_actor_id = {id12}
        OR new_actor_id = {id13} OR new_actor_id = {id14} OR new_actor_id = {id15} OR new_actor_id = {id16}
        OR new_actor_id = {id17} OR new_actor_id = {id18} OR new_actor_id = {id19} OR new_actor_id = {id20}
'''.format(
    TEMP_EVAL_TABLE1=TEMP_FOLLOW_TABLE5,
    TEMP_EVAL_TABLE2=TEMP_EVAL_TABLE5,
    id1=burstbirank_actor_top20[0],
    id2=burstbirank_actor_top20[1],
    id3=burstbirank_actor_top20[2],
    id4=burstbirank_actor_top20[3],
    id5=burstbirank_actor_top20[4],
    id6=burstbirank_actor_top20[5],
    id7=burstbirank_actor_top20[6],
    id8=burstbirank_actor_top20[7],
    id9=burstbirank_actor_top20[8],
    id10=burstbirank_actor_top20[9],
    id11=burstbirank_actor_top20[10],
    id12=burstbirank_actor_top20[11],
    id13=burstbirank_actor_top20[12],
    id14=burstbirank_actor_top20[13],
    id15=burstbirank_actor_top20[14],
    id16=burstbirank_actor_top20[15],
    id17=burstbirank_actor_top20[16],
    id18=burstbirank_actor_top20[17],
    id19=burstbirank_actor_top20[18],
    id20=burstbirank_actor_top20[19]
)
table = o.execute_sql(sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [162]:
data = DataFrame(o.get_table(TEMP_EVAL_TABLE5))
data.head(10)

,new_actor_id,follower_rank
0,221311,134012
1,1142911,143070


In [163]:
burstbirank_actor_top20_infollow = data.to_pandas()
actor_result_top20 = pd.DataFrame(actor_result, columns=['new_actor_id','actor_name','rank'])[:20]
# 合并burstbirank_actor_top20_infollow和actor_result_top20
burstbirank_follow_top20 = pd.merge(actor_result_top20, burstbirank_actor_top20_infollow, on='new_actor_id')
burstbirank_follow_top20

,new_actor_id,actor_name,rank,follower_rank
0,1142911,codeserver-test1,5,143070
1,221311,bryant1410,12,134012


##### top50

In [164]:
burstbirank_actor_top50=actor_info['new_actor_id'][:50].tolist()
print(burstbirank_actor_top50)

[414972, 673689, 361520, 334816, 1142911, 595188, 1014261, 571640, 1065468, 315487, 122996, 221311, 1275949, 7479, 344388, 215104, 343522, 1235182, 1237106, 363095, 1021292, 114227, 725883, 1053802, 936214, 892941, 509911, 26910, 166969, 1135345, 59749, 1023804, 294147, 1223383, 1304580, 890869, 948380, 327204, 581408, 166538, 1070116, 950558, 92, 75252, 375568, 174150, 831667, 394489, 178671, 208100]


In [165]:
TEMP_EVAL_TABLE6 = 'hx_burstbirank_follow_actor_top50'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_EVAL_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_EVAL_TABLE}
    (
        new_actor_id     BIGINT,
        follower_rank    BIGINT
    );
'''.format(
    TEMP_EVAL_TABLE=TEMP_EVAL_TABLE6
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520025417612g2h305pr2>

In [166]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_EVAL_TABLE2}
        SELECT new_actor_id, follower_rank
        FROM {TEMP_EVAL_TABLE1}
        WHERE new_actor_id = {id1} OR new_actor_id = {id2} OR new_actor_id = {id3} OR new_actor_id = {id4} 
        OR new_actor_id = {id5} OR new_actor_id = {id6} OR new_actor_id = {id7} OR new_actor_id = {id8}
        OR new_actor_id = {id9} OR new_actor_id = {id10} OR new_actor_id = {id11} OR new_actor_id = {id12}
        OR new_actor_id = {id13} OR new_actor_id = {id14} OR new_actor_id = {id15} OR new_actor_id = {id16}
        OR new_actor_id = {id17} OR new_actor_id = {id18} OR new_actor_id = {id19} OR new_actor_id = {id20}
        OR new_actor_id = {id21} OR new_actor_id = {id22} OR new_actor_id = {id23} OR new_actor_id = {id24}
        OR new_actor_id = {id25} OR new_actor_id = {id26} OR new_actor_id = {id27} OR new_actor_id = {id28}
        OR new_actor_id = {id29} OR new_actor_id = {id30} OR new_actor_id = {id31} OR new_actor_id = {id32}
        OR new_actor_id = {id33} OR new_actor_id = {id34} OR new_actor_id = {id35} OR new_actor_id = {id36}
        OR new_actor_id = {id37} OR new_actor_id = {id38} OR new_actor_id = {id39} OR new_actor_id = {id40}
        OR new_actor_id = {id41} OR new_actor_id = {id42} OR new_actor_id = {id43} OR new_actor_id = {id44}
        OR new_actor_id = {id45} OR new_actor_id = {id46} OR new_actor_id = {id47} OR new_actor_id = {id48}
        OR new_actor_id = {id49} OR new_actor_id = {id50}
'''.format(
    TEMP_EVAL_TABLE1=TEMP_FOLLOW_TABLE5,
    TEMP_EVAL_TABLE2=TEMP_EVAL_TABLE6,
    id1=burstbirank_actor_top50[0], id2=burstbirank_actor_top50[1], id3=burstbirank_actor_top50[2], id4=burstbirank_actor_top50[3],
    id5=burstbirank_actor_top50[4], id6=burstbirank_actor_top50[5], id7=burstbirank_actor_top50[6], id8=burstbirank_actor_top50[7],
    id9=burstbirank_actor_top50[8], id10=burstbirank_actor_top50[9], id11=burstbirank_actor_top50[10], id12=burstbirank_actor_top50[11],
    id13=burstbirank_actor_top50[12], id14=burstbirank_actor_top50[13], id15=burstbirank_actor_top50[14], id16=burstbirank_actor_top50[15],
    id17=burstbirank_actor_top50[16], id18=burstbirank_actor_top50[17], id19=burstbirank_actor_top50[18], id20=burstbirank_actor_top50[19],
    id21=burstbirank_actor_top50[20], id22=burstbirank_actor_top50[21], id23=burstbirank_actor_top50[22], id24=burstbirank_actor_top50[23],
    id25=burstbirank_actor_top50[24], id26=burstbirank_actor_top50[25], id27=burstbirank_actor_top50[26], id28=burstbirank_actor_top50[27],
    id29=burstbirank_actor_top50[28], id30=burstbirank_actor_top50[29], id31=burstbirank_actor_top50[30], id32=burstbirank_actor_top50[31],
    id33=burstbirank_actor_top50[32], id34=burstbirank_actor_top50[33], id35=burstbirank_actor_top50[34], id36=burstbirank_actor_top50[35],
    id37=burstbirank_actor_top50[36], id38=burstbirank_actor_top50[37], id39=burstbirank_actor_top50[38], id40=burstbirank_actor_top50[39],
    id41=burstbirank_actor_top50[40], id42=burstbirank_actor_top50[41], id43=burstbirank_actor_top50[42], id44=burstbirank_actor_top50[43],
    id45=burstbirank_actor_top50[44], id46=burstbirank_actor_top50[45], id47=burstbirank_actor_top50[46], id48=burstbirank_actor_top50[47],
    id49=burstbirank_actor_top50[48], id50=burstbirank_actor_top50[49]
)
table = o.execute_sql(sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [167]:
data = DataFrame(o.get_table(TEMP_EVAL_TABLE6))
data.head(10)

,new_actor_id,follower_rank
0,92,5790
1,166969,35884
2,725883,39463
3,178671,95753
4,221311,134012
5,1142911,143070
6,1304580,153240


In [168]:
burstbirank_actor_top50_infollow = data.to_pandas()
actor_result_top50 = pd.DataFrame(actor_result, columns=['new_actor_id','actor_name','rank'])[:50]
# 合并burstbirank_actor_top50_infollow和actor_result_top50
burstbirank_follow_top50 = pd.merge(actor_result_top50, burstbirank_actor_top50_infollow, on='new_actor_id')
burstbirank_follow_top50

,new_actor_id,actor_name,rank,follower_rank
0,1142911,codeserver-test1,5,143070
1,221311,bryant1410,12,134012
2,725883,0xflotus,23,39463
3,166969,githubteacher,29,35884
4,1304580,githubvaladm,35,153240
5,92,olleolleolle,43,5790
6,178671,fengjixuchui,49,95753


#### correlation

##### repo

burstbirank_watch_top10, burstbirank_watch_top20, burstbirank_watch_top50中，rank与watch_rank的spearman相关系数检验

In [169]:
import scipy.stats
print("burstbirank表前10的repo的排名与其在watch中对应排名的correlation值：",
      scipy.stats.spearmanr(burstbirank_watch_top10['rank'].to_list(), burstbirank_watch_top10['watch_rank'].to_list())[0])
print("burstbirank表前20的repo的排名与其在watch中对应排名的correlation值：",
      scipy.stats.spearmanr(burstbirank_watch_top20['rank'].to_list(), burstbirank_watch_top20['watch_rank'].to_list())[0])
print("burstbirank表前50的repo的排名与其在watch中对应排名的correlation值：",
      scipy.stats.spearmanr(burstbirank_watch_top50['rank'].to_list(), burstbirank_watch_top50['watch_rank'].to_list())[0])

burstbirank表前10的repo的排名与其在watch中对应排名的correlation值： -0.11515151515151514
burstbirank表前20的repo的排名与其在watch中对应排名的correlation值： -0.31127819548872177
burstbirank表前50的repo的排名与其在watch中对应排名的correlation值： 0.1291236494597839


由于correlation在(-0.2, 0.2)之间，故相关性弱。

burstbirank得到的排名靠前的repo排名与watch统计信息中的排名几乎不相关，二者结果相差较大

##### actor

In [170]:
print("burstbirank表前10的actor的排名与其在follow中对应排名的correlation值：",
      scipy.stats.spearmanr(burstbirank_follow_top10['rank'].to_list(), burstbirank_follow_top10['follower_rank'].to_list())[0])
print("burstbirank表前20的actor的排名与其在follow中对应排名的correlation值：",
      scipy.stats.spearmanr(burstbirank_follow_top20['rank'].to_list(), burstbirank_follow_top20['follower_rank'].to_list())[0])
print("burstbirank表前50的actor的排名与其在follow中对应排名的correlation值：",
      scipy.stats.spearmanr(burstbirank_follow_top50['rank'].to_list(), burstbirank_follow_top50['follower_rank'].to_list())[0])

burstbirank表前10的actor的排名与其在follow中对应排名的correlation值： nan
burstbirank表前20的actor的排名与其在follow中对应排名的correlation值： -0.9999999999999999
burstbirank表前50的actor的排名与其在follow中对应排名的correlation值： -0.3571428571428572


与repo情况相似，burstbirank得到的排名靠前的actor排名与follower统计信息中的排名几乎不相关

### 前1000

In [171]:
# burstbirank
burstbirank_repo_top1000 = pd.DataFrame(repo_result, columns=['new_repo_id'])[:1000]
burstbirank_repo_top1000.to_csv('./burstbirank_repo_top1000.csv',index=False)
burstbirank_actor_top1000 = pd.DataFrame(actor_result, columns=['new_actor_id'])[:1000]
burstbirank_actor_top1000.to_csv('./burstbirank_actor_top1000.csv',index=False)

In [172]:
#watch
burstbirank_watch_top1000 = pd.DataFrame(pd_idwatch, columns=['new_repo_id'])[:1000]
burstbirank_watch_top1000.to_csv('./burstbirank_watch_top1000.csv',index=False)
#follow
burstbirank_follow_top1000 = pd.DataFrame(pd_idfollow, columns=['new_actor_id'])[:1000]
burstbirank_follow_top1000.to_csv('./burstbirank_follow_top1000.csv',index=False)

### birank/pagerank的new_id

In [173]:
TEMP_BIRANK_TABLE0 = 'hx_repo_actor_birank_pull_id_and_name'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_BIRANK_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_BIRANK_TABLE}
    (
        repo_id     BIGINT,
        repo_name    STRING,
        actor_id    BIGINT,
        actor_login    STRING,
        created_at    DATETIME
    );
'''.format(
    TEMP_BIRANK_TABLE=TEMP_BIRANK_TABLE0
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520025508393gossg4pr2>

In [174]:
# 把时间改掉
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_BIRANK_TABLE}
        SELECT
            repo_id,
            repo_name,
            actor_id,
            actor_login,
            created_at
        FROM ods_github_log
        WHERE (type = 'PullRequestEvent') AND (action = 'opened') AND actor_login NOT LIKE '%bot%'
        AND pt>='20160101' AND pt<'20220101'
        ORDER BY repo_id, actor_id'''.format(
    TEMP_BIRANK_TABLE=TEMP_BIRANK_TABLE0)
table = o.execute_sql(sql, hints={"odps.sql.submit.mode" : "script"})

In [175]:
data = DataFrame(o.get_table(TEMP_BIRANK_TABLE0))
data.head(10)

,repo_id,repo_name,actor_id,actor_login,created_at
0,1,mojombo/grit,24193253,ankitts12,2020-10-01 11:55:32
1,27,rubinius/rubinius,85,brixen,2016-02-06 02:05:54
2,27,rubinius/rubinius,85,brixen,2016-02-05 02:38:10
3,27,rubinius/rubinius,85,brixen,2016-04-22 14:31:58
4,27,rubinius/rubinius,85,brixen,2016-04-25 05:05:13
5,27,rubinius/rubinius,85,brixen,2016-06-19 04:39:25
6,27,rubinius/rubinius,85,brixen,2016-06-05 02:04:47
7,27,rubinius/rubinius,85,brixen,2016-05-06 07:47:28
8,27,rubinius/rubinius,85,brixen,2016-05-21 07:27:56
9,27,rubinius/rubinius,85,brixen,2016-05-01 14:19:25


#### 重新编号

##### repo_id重新编号

In [176]:
TEMP_BIRANK_TABLE2 = 'hx_repo_actor_birank_pull_new_repoid_pre'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_BIRANK_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_BIRANK_TABLE}
    (
        repo_id     BIGINT
    );
'''.format(
    TEMP_BIRANK_TABLE=TEMP_BIRANK_TABLE2
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520030002246g5j0igr>

In [178]:
insert_new_repoid_pre_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_BIRANK_TABLE2}
        SELECT DISTINCT repo_id
        FROM {TEMP_BIRANK_TABLE1}
        ORDER BY repo_id
'''.format(
    TEMP_BIRANK_TABLE1=TEMP_BIRANK_TABLE0,
    TEMP_BIRANK_TABLE2=TEMP_BIRANK_TABLE2
)
table = o.execute_sql(insert_new_repoid_pre_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [179]:
data = DataFrame(o.get_table(TEMP_BIRANK_TABLE2))
data.head(15)

,repo_id
0,1
1,27
2,28
3,31
4,35
5,43
6,68
7,98
8,119
9,129


In [180]:
TEMP_BIRANK_TABLE3 = 'hx_repo_actor_birank_pull_new_repoid'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_BIRANK_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_BIRANK_TABLE}
    (
        repo_id     BIGINT,
        new_repo_id    BIGINT
    );
'''.format(
    TEMP_BIRANK_TABLE=TEMP_BIRANK_TABLE3
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520030044992g1xe1ii5>

In [181]:
insert_new_repoid_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_BIRANK_TABLE2}
        SELECT repo_id, row_number() over() - 1 as new_repo_id
        FROM {TEMP_BIRANK_TABLE1}
        ORDER BY repo_id
'''.format(
    TEMP_BIRANK_TABLE1=TEMP_BIRANK_TABLE2,
    TEMP_BIRANK_TABLE2=TEMP_BIRANK_TABLE3
)
table = o.execute_sql(insert_new_repoid_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [182]:
data = DataFrame(o.get_table(TEMP_BIRANK_TABLE3))
data.head(15)

,repo_id,new_repo_id
0,1,0
1,27,1
2,28,2
3,31,3
4,35,4
5,43,5
6,68,6
7,98,7
8,119,8
9,129,9


In [183]:
# 查看最大new_repo_id(用于保证与WATCH中的new_repo_id一致)
select_max_new_repo_id = '''SELECT MAX(new_repo_id) FROM {TEMP_BIRANK_TABLE}'''.format(TEMP_BIRANK_TABLE=TEMP_BIRANK_TABLE3)
table_max_new_repo_id = o.execute_sql(select_max_new_repo_id, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})
with table_max_new_repo_id.open_reader(tunnel=True) as reader:
     for record in reader:
            print(record)

odps.Record {
  _c0   14800777
}


##### actor_id重新编号

In [184]:
TEMP_BIRANK_TABLE4 = 'hx_repo_actor_birank_pull_new_actorid_pre'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_BIRANK_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_BIRANK_TABLE}
    (
        actor_id     BIGINT
    );
'''.format(
    TEMP_BIRANK_TABLE=TEMP_BIRANK_TABLE4
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520030107346g2v6uq2>

In [185]:
insert_new_actorid_pre_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_BIRANK_TABLE2}
        SELECT DISTINCT actor_id
        FROM {TEMP_BIRANK_TABLE1}
        ORDER BY actor_id
'''.format(
    TEMP_BIRANK_TABLE1=TEMP_BIRANK_TABLE0,
    TEMP_BIRANK_TABLE2=TEMP_BIRANK_TABLE4
)
table = o.execute_sql(insert_new_actorid_pre_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [186]:
data = DataFrame(o.get_table(TEMP_BIRANK_TABLE4))
data.head(15)

,actor_id
0,1
1,2
2,4
3,6
4,7
5,17
6,18
7,19
8,20
9,21


In [190]:
TEMP_BIRANK_TABLE5 = 'hx_repo_actor_birank_pull_new_actorid'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_BIRANK_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_BIRANK_TABLE}
    (
        actor_id     BIGINT,
        new_actor_id    BIGINT
    );
'''.format(
    TEMP_BIRANK_TABLE=TEMP_BIRANK_TABLE5
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 2022052003070174gchykkm7>

In [191]:
insert_new_actorid_sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_BIRANK_TABLE2}
        SELECT actor_id, row_number() over() - 1 as new_actor_id
        FROM {TEMP_BIRANK_TABLE1}
        ORDER BY actor_id
'''.format(
    TEMP_BIRANK_TABLE1=TEMP_BIRANK_TABLE4,
    TEMP_BIRANK_TABLE2=TEMP_BIRANK_TABLE5
)
table = o.execute_sql(insert_new_actorid_sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [192]:
data = DataFrame(o.get_table(TEMP_BIRANK_TABLE5))
data.head(15)

,actor_id,new_actor_id
0,1,0
1,2,1
2,4,2
3,6,3
4,7,4
5,17,5
6,18,6
7,19,7
8,20,8
9,21,9


In [193]:
# 查看最大new_actor_id(用于保证与follower中的new_actor_id一致)
select_max_new_actor_id = '''SELECT MAX(new_actor_id) FROM {TEMP_BIRANK_TABLE}'''.format(TEMP_BIRANK_TABLE=TEMP_BIRANK_TABLE5)
table_max_new_actor_id = o.execute_sql(select_max_new_actor_id, hints={'odps.sql.allow.fullscan': 'true', "odps.sql.submit.mode" : "script"})
with table_max_new_actor_id.open_reader(tunnel=True) as reader:
     for record in reader:
            print(record)

odps.Record {
  _c0   7282527
}


#### 连接burstbirank的new_id---id---birank的new_id

TEMP_TABLE_NAME8为burstbirank_new_repo_id---repo_id

TEMP_BIRANK_TABLE3为birank_new_repo_id---repo_id

TEMP_TABLE_NAME10为burstbirank_new_actor_id---actor_id

TEMP_BIRANK_TABLE5为birank_new_actor_id---actor_id

In [194]:
TEMP_BURST_TABLE1 = 'hx_repo_actor_pull_request_newnewrepoid'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_BIRANK_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_BIRANK_TABLE}
    (
        new_repo_id     BIGINT,
        new_new_repo_id    BIGINT
    );
'''.format(
    TEMP_BIRANK_TABLE=TEMP_BURST_TABLE1
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 2022052003072641g8d6k963>

In [195]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_BURST_TABLE3}
        SELECT {TEMP_BURST_TABLE1}.new_repo_id, {TEMP_BURST_TABLE2}.new_repo_id
        FROM {TEMP_BURST_TABLE1}, {TEMP_BURST_TABLE2}
        WHERE {TEMP_BURST_TABLE1}.repo_id = {TEMP_BURST_TABLE2}.repo_id
        ORDER BY {TEMP_BURST_TABLE1}.new_repo_id
'''.format(
    TEMP_BURST_TABLE1=TEMP_TABLE_NAME8,
    TEMP_BURST_TABLE2=TEMP_BIRANK_TABLE3,
    TEMP_BURST_TABLE3=TEMP_BURST_TABLE1
)
table = o.execute_sql(sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [196]:
data = DataFrame(o.get_table(TEMP_BURST_TABLE1))
data.head(15)

,new_repo_id,new_new_repo_id
0,0,0
1,1,1
2,2,2
3,3,4
4,4,5
5,5,6
6,6,10
7,7,11
8,8,12
9,9,13


In [197]:
pd_newnewrepoid = data.to_pandas()
pd_newnewrepoid.head()

,new_repo_id,new_new_repo_id
0,0,0
1,1,1
2,2,2
3,3,4
4,4,5


In [198]:
TEMP_BURST_TABLE2 = 'hx_repo_actor_pull_request_newnewactorid'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_BIRANK_TABLE};
    CREATE TABLE IF NOT EXISTS {TEMP_BIRANK_TABLE}
    (
        new_actor_id     BIGINT,
        new_new_actor_id    BIGINT
    );
'''.format(
    TEMP_BIRANK_TABLE=TEMP_BURST_TABLE2
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220520031003572gpbfc4pr2>

In [199]:
sql = '''set odps.sql.validate.orderby.limit=false;
        INSERT INTO {TEMP_BURST_TABLE3}
        SELECT {TEMP_BURST_TABLE1}.new_actor_id, {TEMP_BURST_TABLE2}.new_actor_id
        FROM {TEMP_BURST_TABLE1}, {TEMP_BURST_TABLE2}
        WHERE {TEMP_BURST_TABLE1}.actor_id = {TEMP_BURST_TABLE2}.actor_id
        ORDER BY {TEMP_BURST_TABLE1}.new_actor_id
'''.format(
    TEMP_BURST_TABLE1=TEMP_TABLE_NAME10,
    TEMP_BURST_TABLE2=TEMP_BIRANK_TABLE5,
    TEMP_BURST_TABLE3=TEMP_BURST_TABLE2
)
table = o.execute_sql(sql, hints={'odps.sql.allow.fullscan': 'true',"odps.sql.submit.mode" : "script"})

In [200]:
data = DataFrame(o.get_table(TEMP_BURST_TABLE2))
data.head(15)

,new_actor_id,new_new_actor_id
0,0,0
1,1,2
2,2,4
3,3,5
4,4,6
5,5,7
6,6,8
7,7,9
8,8,10
9,9,12


In [201]:
pd_newnewactorid = data.to_pandas()
pd_newnewactorid.head()

,new_actor_id,new_new_actor_id
0,0,0
1,1,2
2,2,4
3,3,5
4,4,6


#### new_new_id---rank

repo:

pd_newnewrepoid为new_repo_id---new_new_repo_id

repo_result有new_repo_id---burstbirank_repo_rank

actor:

pd_newnewactorid为new_actor_id---new_new_actor_id

actor_result有new_actor_id---burstbirank_actor_rank

In [209]:
burstbirank_repo_rank_top1000 = pd.DataFrame(repo_result, columns=['new_repo_id','rank'])[:1000]
new_new_burstbirank_repo_rank_top1000_pre = pd.merge(burstbirank_repo_rank_top1000, pd_newnewrepoid, on = 'new_repo_id')
new_new_burstbirank_repo_rank_top1000 = pd.DataFrame(new_new_burstbirank_repo_rank_top1000_pre, columns=['new_new_repo_id']).rename(columns={'new_new_repo_id':'new_repo_id'})
new_new_burstbirank_repo_rank_top1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   new_repo_id  1000 non-null   int64
dtypes: int64(1)
memory usage: 15.6 KB


In [210]:
burstbirank_actor_rank_top1000 = pd.DataFrame(actor_result, columns=['new_actor_id','rank'])[:1000]
new_new_burstbirank_actor_rank_top1000_pre = pd.merge(burstbirank_actor_rank_top1000, pd_newnewactorid, on = 'new_actor_id')
new_new_burstbirank_actor_rank_top1000 = pd.DataFrame(new_new_burstbirank_actor_rank_top1000_pre, columns=['new_new_actor_id']).rename(columns={'new_new_actor_id':'new_actor_id'})
new_new_burstbirank_actor_rank_top1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   new_actor_id  1000 non-null   int64
dtypes: int64(1)
memory usage: 15.6 KB


In [212]:
new_new_burstbirank_repo_rank_top1000.to_csv('./burstbirank_repo_top1000.csv')
new_new_burstbirank_actor_rank_top1000.to_csv('./burstbirank_actor_top1000.csv')